COVID Subgroup Analysis

In [ ]:
import os
from os import path, makedirs
import sys
import copy
import pickle
from collections import namedtuple, defaultdict, Counter
from datetime import datetime, timedelta, date
from time import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, transforms as mtransforms
from random import Random
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.cluster import KMeans
import getpass
import pyodbc
import sqlalchemy
from scipy.stats import chi2_contingency, mannwhitneyu
from IPython.display import Markdown, display

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)

def printmd(string):
    display(Markdown(string))

In [ ]:
dir_data = '/path/to/data/dir'
dir_model = '/path/to/model/dir'
# input
file_pcs = path.join(dir_data, 'patient_code_sequences.txt')
file_events = path.join(dir_data, 'events_covid19.csv')
file_concepts = path.join(dir_data, 'concepts.csv')
file_model_dm = path.join(dir_model, 'doc2vec_dm_model.d2v')
file_model_dbow = path.join(dir_model, 'doc2vec_dbow_model.d2v')

# output
timestamp_str = datetime.now().strftime('%y%m%d_%H%M')
dir_output = path.join(dir_data, timestamp_str)
makedirs(dir_output)
file_backup_suffix = '.backup'


In [ ]:
pwd=getpass.getpass()

In [ ]:
# SQL server config
sql_config = {
    'driver': 'ODBC Driver 17 for SQL Server',
    'server': 'sql.server.host',
    'database': 'database_name',
    'uid': 'user_name'
}

conn = pyodbc.connect(**sql_config, pwd=pwd)
cursor = conn.cursor()

In [ ]:
connection_url = sqlalchemy.engine.URL.create(
    "mssql+pyodbc",
    username="username",
    password=pwd,
    host="sql.server.host",
    database="database_name",
    query={
        "driver": "ODBC Driver 17 for SQL Server"
    },
)
engine = sqlalchemy.create_engine(connection_url)

In [ ]:
time_window_covid = [0,28]
time_window_pre = [-730, -22]

## Load data into dataframe

In [ ]:
# Load the concept definitions
df_concepts = pd.read_csv(file_concepts, sep='\t', header=0, index_col='concept_id', low_memory=False)
df_concepts.head()

In [ ]:
# Load the index events
df_events_orig = pd.read_csv(file_events, sep='\t', header=0, index_col='person_id')
df_events_orig['start_date'] = df_events_orig['start_date'].apply(date.fromisoformat)
df_events_orig['end_date'] = df_events_orig['end_date'].apply(date.fromisoformat)
df_events_orig['op_start_date'] = df_events_orig['op_start_date'].apply(date.fromisoformat)
df_events_orig['op_end_date'] = df_events_orig['op_end_date'].apply(date.fromisoformat)
df_events_orig.sort_values(by='start_date', inplace=True)

print(len(df_events_orig))
df_events_orig.head()

# Load the PV models

In [ ]:
model_dm = Doc2Vec.load(file_model_dm)

In [ ]:
model_dbow = Doc2Vec.load(file_model_dbow)

# Build the cohort for disease subtyping

In [ ]:
# Helpers for reading in the patient_code_sequences.txt

# Date of occurrence and list of concepts occurring on this date
DateOccurrence = namedtuple('DateOccurrence', ['date', 'concept_ids'])

def _process_pcs_line(line):
    """ Processes a line from patient_code_sequences.txt and parses out the patient ID
    and DateOccurrences """
    split = line.strip().split('\t')
        
    # person_id is the first entry
    pid = int(split.pop(0))
    
    # Process the remaining string into a list of Occurrences
    date_occurrences = [_process_date_occurrence_str(x) for x in split]
    
    return pid, date_occurrences

def _process_date_occurrence_str(dos):
    """ Processes a DateOccurrence string 
    format: YYYY-MM-DD:<list of concept IDs separated by commas> """
    date_str, concept_ids_str = dos.split(':')
    occ = DateOccurrence(date.fromisoformat(date_str), 
                         [int(x) for x in concept_ids_str.split(',')])
    return occ

def create_patient_sequences(f_pcs_in, f_seq_out=None, min_seq_length=10, randomize_order=True, random_seed=None, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and parses it into sequences for each patient
    
    Note: save_intermediates makes it a lot slower """

    # For keeping track of processing time
    t1 = time()

    # pseqs - list of TaggedDocument(words=[concept_ids], tags=[person_id])
    pseqs = list()

    count = 0
    
    r = Random(random_seed)
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the heaer line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)

            # Combine sequence of concepts from each date into on sequence for the patient
            current_seq = []
            for date_occurrence in date_occurrences:
                concepts = date_occurrence.concept_ids
                if randomize_order:
                    # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                    r.shuffle(concepts)
                    
                current_seq += concepts
                
            if len(current_seq) >= min_seq_length:
                pseqs.append(TaggedDocument(words=[str(x) for x in current_seq], tags=[pid]))

            # Display progress
            count += 1
            if count % 100000 == 0:
                if verbose: 
                    # Processing time and size of data structure
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(pseqs, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save the concept age distributions            
        pickle.dump(pseqs, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return pseqs

# Class for storing info about a cohort patient sequence
# Sequence stored as TaggedDocument object for D2V processing
# Will use OMOP concept ID for label
CohortPatientSeq = namedtuple('CohortPatientSeq', ['person_id', 'sequence', 'label', 'date_lower', 'date_upper'])

def create_cohort_patient_sequences(f_pcs_in, df_events, cohort_concept=37311061, f_seq_out=None, inclusion_window=(None, None), time_window=[-14,28], event_end=False, 
                                    randomize_order=True, random_seed=None, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and extracts sequences within the time_window days around the 
    first occurrence of any encountered desired concept
    
    Parameters
    ----------
    f_pcs_in: filename of patient code sequences file
    df_events: dataframe of events
    cohort_concpets: not really used for COVID-19 analysis; default 37311061
    f_seq_out: filename to write sequences to
    inclusion_window: range of event start dates that are included: (date range beginning, date range ending). None implies no restrictions applied. Default: (None, None)
    time_window: relative date range to include for each patient, relative to index date (hospital start date): [date range beginnning, date range ending]
    event_end: If true, will use the max(event end date, time_window[1]) as the end date. If false, will use time_window[1] as the end date.
    randomize_order: If true, randomizes order of codes per day
    verbose: If true, prints verbose progress
    save_intermediate: save intermediate work files. Note: this makes it a lot slower 
    
    Return 
    ------
    cpss: list[CohortPatientSequence]"""

    # For keeping track of processing time
    t1 = time()

    # pcss - cohort patient sequences: list of CohortPatientSeq objects
    cpss = dict()
    count = 0
    
    # Time window for finding occurrences 
    time_window_pre = timedelta(days=time_window[0])
    time_window_post = timedelta(days=time_window[1])
    
    # Event inclusion window
    if inclusion_window is None or len(inclusion_window) != 2:
        print('Warning, inclusion window expected to be tuple of length 2. Proceeding with default value.')
        inclusion_window = (None, None)
    else:
        inclusion_window = tuple([date.fromisoformat(d) if type(d) is str else d for d in inclusion_window])        
    
    r = Random(random_seed)
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the header line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)
            
            if pid not in df_events.index:
                # Couldn't find this person_id in the events table
                print(f"Could not find person_id {pid} in df_events")
                continue
                
            # Get the index date from events table    
            event_start_date = df_events.loc[pid, 'start_date']
            event_end_date = df_events.loc[pid, 'end_date']

            # Check event date inclusion criteria
            if ((inclusion_window[0] is not None and event_start_date < inclusion_window[0]) or 
                    (inclusion_window[1] is not None and event_start_date > inclusion_window[1])):
                # event was outside inclusion window
                print(f'{event_start_date} outside of inclusion window')
                continue
    
            date_lower = event_start_date + time_window_pre
            date_upper = event_start_date + time_window_post
            if event_end:                                
                # Use the max of the upper limit as specified by the time window or the hospitalization end date
                date_upper = max(date_upper, event_end_date)
            
            current_seq = list()
            for do in date_occurrences:
                if do.date < date_lower:
                    continue

                if do.date > date_upper:
                    # No more date_occurrences within the desired time window.                         
                    break

                # The date_occurrence is within the time_window. Add occurrences to seq
                concepts = do.concept_ids
                if randomize_order:
                    # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                    r.shuffle(concepts)
                current_seq += concepts

            # Convert the sequence of OMOP concept IDs to TaggedDocument for D2V processing
            tagged_doc_seq = TaggedDocument(words=[str(x) for x in current_seq], tags=[pid])                

            # Save the sequence along with patient ID, time window, and label
            cps = CohortPatientSeq(pid, tagged_doc_seq, cohort_concept, date_lower, date_upper)
            cpss[pid] = cps

            # Display progress
            count += 1
            if count % 1000 == 0:
                if verbose: 
                    # Processing time
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(cpss, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save         
        pickle.dump(cpss, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return cpss

### COVID-19

37311061 - Disease caused by severe acute respiratory syndrome coronavirus 2

In [ ]:
file_sequences = path.join(dir_output, 'covid_cohort_inpatient_sequences.pkl')
cpss_orig = create_cohort_patient_sequences(file_pcs, df_events_orig, f_seq_out=file_sequences, inclusion_window=('2020-03-01', None), time_window=time_window_covid, 
                                            event_end=True, randomize_order=True, random_seed=42, verbose=True, save_intermediates=False)
n_cpss_orig = len(cpss_orig)
print(n_cpss_orig)

Also get patient sequences prior to their COVID diagnosis so that we can evaluate their baseline

In [ ]:
file_sequences_pre = path.join(dir_output, 'covid_cohort_inpatient_sequences_pre.pkl')
cpss_pre_orig = create_cohort_patient_sequences(file_pcs, df_events_orig, f_seq_out=file_sequences_pre, time_window=time_window_pre, 
                                       randomize_order=True, random_seed=42, verbose=True, save_intermediates=False)
n_cpss_pre_orig = len(cpss_pre_orig)
print(n_cpss_pre_orig)

In [ ]:
# Count the number of distinct patients
patient_ids_orig = list(cpss_orig.keys())
n_patients_orig = len(patient_ids_orig)
print(n_patients_orig)

In [ ]:
# Trim down df_events to the patients that we have data for
print(len(df_events_orig))
df_events_orig = df_events_orig[df_events_orig.index.isin(patient_ids_orig)]
print(len(df_events_orig))

### Characterize the patients

In [ ]:
def demographics(df_events):
    sql = f"""
        SELECT c_g.concept_name AS sex, convert(date, birth_datetime) AS birth_date, 
            CASE 
                WHEN race_concept_id IN (
                    8557, -- Native Hawaiian or Other Pacific Islander
                    8657, -- American Indian or Alaska Native                    
                    8515 -- Asians
                    ) THEN 'Asian, AIAN, or NHPI'
                WHEN race_concept_id NOT IN (
                    0, -- no matching concept
                    8522, -- other race
                    8552, -- unknown
                    44814653 -- unknown
                    ) THEN c_r.concept_name                                             
                WHEN ethnicity_concept_id = 38003563 THEN 'Hispanic'
                WHEN ethnicity_concept_id IN (
                    0, -- no matching concept
                    38003564 -- not hispanic or latino
                    ) THEN 'Other or Unknown'
                ELSE 
                    -- Red flag to indicate we didn't account for some combination
                    CONCAT(str(race_concept_id), '-', str(ethnicity_concept_id))
            END AS ethnicity_race      
        FROM person p 
        JOIN concept c_g ON p.gender_concept_id = c_g.concept_id
        JOIN concept c_r ON p.race_concept_id = c_r.concept_id
        WHERE person_id = ?
        """    
    data_demogs = list()
    for person_id, row in df_events.iterrows():
        df = cursor.execute(sql, person_id)          
        patient_demo_row = cursor.fetchone()                        
        age = (row['start_date'] - patient_demo_row[1]).days / 365.2425  # Calculate age at event
        data_demogs.append([patient_demo_row[0], age, patient_demo_row[2]])                
                        
    return pd.DataFrame(data_demogs, columns=['sex', 'age', 'race_ethnicity'], index=df_events.index)              

def visit_characteristics(df_events, time_window_pre_covid, time_window_covid):
    # Count how many visits each patient had in the pre-COVID and COVID time windows
    sql_visit_count = f"""SELECT COUNT(*) AS visit_count
    FROM visit_occurrence
    WHERE person_id = ? AND visit_start_date BETWEEN ? AND ?;
    """    
    data_visits = list()
    for person_id, row in df_events.iterrows():
        event_start_date = row['start_date']        

        # Count pre-COVID visits
        date_lower_pre = event_start_date + timedelta(days=time_window_pre_covid[0])
        date_upper_pre = event_start_date + timedelta(days=time_window_pre_covid[1])
        cursor.execute(sql_visit_count, [person_id, date_lower_pre, date_upper_pre])
        count_pre = cursor.fetchone()[0]

        # Count COVID visits
        date_lower_covid = event_start_date + timedelta(days=time_window_covid[0])
        date_upper_covid = event_start_date + timedelta(days=time_window_covid[1])
        cursor.execute(sql_visit_count, [person_id, date_lower_covid, date_upper_covid])
        count_covid = cursor.fetchone()[0]

        data_visits.append([count_pre, count_covid])
    
    df = pd.DataFrame(data_visits, columns=['visit_count_pre_covid', 'visit_count_covid'], index=df_events.index)
    
    df['event_start_date'] = df_events['start_date']
    
    # Calculate the length of stay of the event visit
    df['event_visit_los'] = df_events.apply(lambda x: (x.end_date - x.start_date).days + 1, axis=1)
    
    return df

def vaccination_rate(df_events, show=False):
    # Get percent of persons with full vaccination status before the start of their hospitalization
    n_persons = len(df_events)
    # This table has unique patients who are fully vaccinated and their earliest full vaccination status
    sql = """SELECT * FROM database_name.dbo.covid19_vaccination;"""
    df = pd.read_sql(sql, engine)
    df.set_index('person_id', inplace=True)    
    df = df_events.join(df, rsuffix='_vax')
    df = df[df.end_date_vax < df.start_date]
    n_vax = len(df)
    p_vax = n_vax / n_persons * 100
    if show:
        print(f'Vaccination rate: {n_vax} / {n_persons} ({p_vax}%)')
    return p_vax

def plot_sex(sex, sex_bg=None, show=True, show_ylabel=True):    
    width = 0.7
    offset = 0
    x = np.array(range(2))
    if sex_bg is not None:
        width = 0.35
        offset = width / 2       
    
    df = pd.DataFrame(sex.value_counts(normalize=True)*100)    
    
    if sex_bg is not None:
        df_bg = pd.DataFrame(sex_bg.value_counts(normalize=True)*100)
        df_bg.columns = ['full']
        df.columns = ['cluster']
        df = df.join(df_bg)
        
    ax = plt.gca()
    df.plot(kind='bar', color=['#1F77B4', '#AAAAAA'], ax=ax)
    plt.xticks(rotation=0)
    
    if show_ylabel:
        plt.ylabel('Patients (%)')
    ax.get_legend().remove()  
    plt.xlabel('Sex')
    
    if show:
        plt.show()
        
def plot_race(race, race_bg=None, show=True, show_ylabel=True):    
    width = 0.7
    offset = 0
    x = np.array(range(2))
    if race_bg is not None:
        width = 0.35
        offset = width / 2       
    
    name_hispanic = 'Hispanic'
    name_asian_aian_nhpi = 'Asian\nAIAN\nNHPI'
    name_black = 'Black'
    name_white ='White'
    name_unknown = 'Other\nUnknown'    
    df = pd.DataFrame(race.value_counts(normalize=True)*100)    
    df = df.rename(index={
        'Other or Unknown': name_unknown,
        'Black or African American': name_black,
        'Asian, AIAN, or NHPI': name_asian_aian_nhpi
    }).reindex([name_hispanic, name_asian_aian_nhpi, name_black, name_white, name_unknown])
    
    if race_bg is not None:
        df_bg = pd.DataFrame(race_bg.value_counts(normalize=True)*100)
        df_bg = df_bg.rename(index={
            'Other or Unknown': name_unknown,
            'Black or African American': name_black,
            'Asian, AIAN, or NHPI': name_asian_aian_nhpi
        }).reindex([name_hispanic, name_asian_aian_nhpi, name_black, name_white, name_unknown])
        df_bg.columns = ['full']
        df.columns = ['cluster']
        df = df.join(df_bg)
        
    ax = plt.gca()
    df.plot(kind='bar', color=['#1F77B4', '#AAAAAA'], ax=ax)
    plt.xticks(rotation=90)
    
    if show_ylabel:
        plt.ylabel('Patients (%)')
    ax.get_legend().remove()  
    plt.xlabel('Race-ethnicity')
    
    if show:
        plt.show()        

def plot_age_histogram(age, age_bg=None, bins=None, ranged_bin_labels=False, skip_xticks=False, show=True):
    if bins is None:
        bins = np.append(np.arange(17)*5, np.Inf)

    # Create bin labels
    
    if ranged_bin_labels:
        # Ranged bin labels
        bin_labels = list()
        for i in range(len(bins) - 1):
            lower = bins[i]
            upper = bins[i+1]
            if lower < upper - 1:
                if upper == np.Inf:
                    bin_labels.append(f'{lower:.0f}+')
                else:
                    bin_labels.append(f'{lower:.0f}-{(upper-1):.0f}')
            else:
                bin_labels.append(f'{lower:.0f}')
    else:
        # Bin labels using just the left edge (more compact labels)
        bin_labels = [f'{x:.0f}' for x in bins][:-1]
        bin_labels[-1] += '+'        
            
    width = 0.7
    offset = 0
    x = np.array(range(len(bins)-1))
    if age_bg is not None:
        width = 0.35
        offset = width / 2   
        
    hist, bin_edges = np.histogram(age, bins=bins)
    plt.bar(x=x-offset, height=hist/len(age)*100, tick_label=bin_labels, width=width)
    plt.xlabel('Age (years)')
    plt.ylabel('Patients (%)')
    
    xticks=x
    if skip_xticks:
        xticks = xticks[0:len(bin_labels):2]
        bin_labels = bin_labels[0:len(bin_labels):2]
    plt.xticks(xticks, labels=bin_labels)
    
    if age_bg is not None:
        hist, bin_edges = np.histogram(age_bg, bins=bins)
        plt.bar(x=x+offset, height=hist/len(age_bg)*100, color='gray', width=width)  
        plt.legend(['cluster', 'full'], loc='best')
    
    if show:
        plt.show()

def plot_visit_counts(visit_counts, visit_counts_bg=None, bins=None, xlabel='Number of Visits', show=True, show_ylabel=True):
    if bins is None:
        bins = [0, 1, 3, 5, 10, 20, 50, 100, np.Inf]

    # Create bin labels
    bin_labels = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i+1]
        if lower < upper - 1:
            if upper == np.Inf:
                bin_labels.append(f'{lower}+')
            else:
                bin_labels.append(f'{lower}-{upper-1}')
        else:
            bin_labels.append(str(lower))
            
    width = 0.7
    offset = 0
    x = np.array(range(len(bins)-1))
    if visit_counts_bg is not None:
        width = 0.35
        offset = width / 2   
            
    hist, bin_edges = np.histogram(visit_counts, bins=bins)
    plt.bar(x=x-offset, height=hist/len(visit_counts)*100, tick_label=bin_labels, width=width)
    plt.xlabel(xlabel)
    if show_ylabel:
        plt.ylabel('Patients (%)')
    plt.xticks(x, rotation=90)
    
    if visit_counts_bg is not None:
        hist, bin_edges = np.histogram(visit_counts_bg, bins=bins)
        plt.bar(x=x+offset, height=hist/len(visit_counts_bg)*100, color='#AAAAAA', width=width)    
        plt.legend(['cluster', 'full'], loc='best')
        
    if show:
        plt.show()
        
    return hist, bin_edges
    
def plot_precovid_visit_counts(visit_counts, visit_counts_bg=None, show=True, show_ylabel=True):
    bins = [0, 1, 3, 5, 10, 20, 50, 100, np.Inf]
    return plot_visit_counts(visit_counts, visit_counts_bg, bins=bins, xlabel='Pre-COVID number of visits', show=show, show_ylabel=show_ylabel)

def plot_covid_visit_counts(visit_counts, visit_counts_bg=None, show=True, show_ylabel=True):
    bins = [1, 2, 3, 5, 7, 10, np.Inf]
    return plot_visit_counts(visit_counts, visit_counts_bg, bins=bins, xlabel='COVID number of visits', show=show, show_ylabel=show_ylabel)
    
def plot_los(los, los_bg=None, bins=None, show=True, show_ylabel=True):                
    if bins is None:
            bins = [1, 3, 5, 10, 20, 50, 100, np.Inf]
            
    # Create bin labels
    bin_labels = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i+1]
        if lower < upper - 1:
            if upper == np.Inf:
                bin_labels.append(f'{lower}+')
            else:
                bin_labels.append(f'{lower}-{upper-1}')
        else:
            bin_labels.append(str(lower))
    
    width = 0.7
    offset = 0
    x = np.array(range(len(bins)-1))
    if los_bg is not None:
        width = 0.35
        offset = width / 2        
    
    hist, bin_edges = np.histogram(los, bins=bins)    
    plt.bar(x=x - offset, height=hist/len(los)*100, width=width, tick_label=bin_labels)
    plt.xlabel('Length of stay (days)')
    if show_ylabel:
        plt.ylabel('Patients (%)')
    plt.xticks(x, rotation=90)
    
    if los_bg is not None:
        hist, bin_edges = np.histogram(los_bg, bins=bins)   
        plt.bar(x=x + offset, height=hist/len(los_bg)*100, width=width, color='#AAAAAA')
        plt.legend(['cluster', 'full'], loc='best')
        
    if show:
        plt.show()
        
def plot_date_by_month(dates, dates_bg=None, start='2020-03', end='2021-11', show=True):
    def get_monthly_stats(dates):
        dates = dates.astype('datetime64')
        n = len(dates)
        # Group by a string: YYYY-MM
        counts = dates.groupby(dates.dt.strftime('%Y-%m')).count()
        # Make sure months with no patients still show 0 (instead of not showing up in the plot)
        counts = counts.reindex(pd.date_range(start, end, freq='MS').strftime("%Y-%m"), fill_value=0)
        percent = counts * 100 / n
        return pd.DataFrame(percent)
        
    percent = get_monthly_stats(dates)
    
    if dates_bg is not None:
        percent_bg = get_monthly_stats(dates_bg)
        percent_bg.columns = ['full']
        percent.columns = ['cluster']
        percent = pd.DataFrame(percent).join(percent_bg)
    
    ax = plt.gca()
    percent.plot(kind='bar', ax=ax, color=['#1F77B4', '#AAAAAA'])
    loc, labels = plt.xticks()
    plt.xticks(loc[0:len(loc):2], labels[0:len(labels):2])
    plt.ylabel('Patients (%)')
    plt.xlabel('Hospitalization month')
    ax.get_legend().remove()
    if show:
        plt.show()
        
def plot_characteristics(df_demographics, df_visit_characteristics, df_demographics_bg=None, df_visit_characteristics_bg=None):
    plt.figure(figsize=(12,12))
    
    plt.subplot(3, 2, 1)
    if df_visit_characteristics_bg is not None:
        plot_date_by_month(df_visit_characteristics.event_start_date, df_visit_characteristics_bg.event_start_date, show=False)
    else:
        plot_date_by_month(df_visit_characteristics.event_start_date, show=False)
        
    plt.subplot(3, 2, 2)
    if df_visit_characteristics_bg is not None:
        plot_los(df_visit_characteristics.event_visit_los, df_visit_characteristics_bg.event_visit_los, show=False)
    else:
        plot_los(df_visit_characteristics.event_visit_los, show=False)
    
    plt.subplot(3, 2, 3)
    if df_demographics_bg is not None:
        plot_age_histogram(df_demographics.age, df_demographics_bg.age, show=False, skip_xticks=True)
    else:
        plot_age_histogram(df_demographics.age, show=False, skip_xticks=True)
    
    plt.subplot(3, 2, 4)
    if df_visit_characteristics_bg is not None:
        plot_precovid_visit_counts(df_visit_characteristics.visit_count_pre_covid, df_visit_characteristics_bg.visit_count_pre_covid, show=False)
    else:
        plot_precovid_visit_counts(df_visit_characteristics.visit_count_pre_covid, show=False)
    
    plt.subplot(3, 2, 5)        
    if df_demographics_bg is not None:
        plot_race(df_demographics.race_ethnicity, df_demographics_bg.race_ethnicity, show=False, show_ylabel=False)
    else:
        plot_race(df_demographics.race_ethnicity, show=False, show_ylabel=False)    
    
    plt.subplot(3, 2, 6)
    if df_visit_characteristics_bg is not None:
        plot_covid_visit_counts(df_visit_characteristics.visit_count_covid, df_visit_characteristics_bg.visit_count_covid, show=False)
    else:
        plot_covid_visit_counts(df_visit_characteristics.visit_count_covid, show=False)
    plt.show()        
    
def plot_characteristics_paper(df_demographics, df_visit_characteristics, df_demographics_bg=None, df_visit_characteristics_bg=None, figsize=(12, 7), file_out=None):
    fig = plt.figure(figsize=figsize)
    translation = mtransforms.ScaledTranslation(8/72, -15/72, fig.dpi_scale_trans)
    
    ax = plt.subplot(2, 3, 1)
    if df_demographics_bg is not None:
        plot_age_histogram(df_demographics.age, df_demographics_bg.age, show=False, skip_xticks=True)
    else:
        plot_age_histogram(df_demographics.age, show=False, skip_xticks=True)
    plt.text(0, 1, '(A)', transform=ax.transAxes + translation, fontweight='bold')
        
    ax = plt.subplot(2, 3, 2)          
    if df_demographics_bg is not None:
        plot_race(df_demographics.race_ethnicity, df_demographics_bg.race_ethnicity, show=False, show_ylabel=False)
    else:
        plot_race(df_demographics.race_ethnicity, show=False, show_ylabel=False)            
    plt.text(0, 1, '(B)', transform=ax.transAxes + translation, fontweight='bold')
        
    ax = plt.subplot(2, 3, 3)
    if df_visit_characteristics_bg is not None:
        plot_los(df_visit_characteristics.event_visit_los, df_visit_characteristics_bg.event_visit_los, show=False, show_ylabel=False)
    else:
        plot_los(df_visit_characteristics.event_visit_los, show=False, show_ylabel=False)     
    plt.text(0, 1, '(C)', transform=ax.transAxes + translation, fontweight='bold')
    
    ax = plt.subplot(2, 3, 4)
    if df_visit_characteristics_bg is not None:
        plot_date_by_month(df_visit_characteristics.event_start_date, df_visit_characteristics_bg.event_start_date, show=False)
    else:
        plot_date_by_month(df_visit_characteristics.event_start_date, show=False)
    plt.text(0.05, 1, '(D)', transform=ax.transAxes + translation, fontweight='bold')
    
    ax = plt.subplot(2, 3, 5)
    if df_visit_characteristics_bg is not None:
        plot_precovid_visit_counts(df_visit_characteristics.visit_count_pre_covid, df_visit_characteristics_bg.visit_count_pre_covid, show=False, show_ylabel=False)
    else:
        plot_precovid_visit_counts(df_visit_characteristics.visit_count_pre_covid, show=False, show_ylabel=False)
    plt.text(0.10, 1, '(E)', transform=ax.transAxes + translation, fontweight='bold')
    
    ax = plt.subplot(2, 3, 6)
    if df_visit_characteristics_bg is not None:
        plot_covid_visit_counts(df_visit_characteristics.visit_count_covid, df_visit_characteristics_bg.visit_count_covid, show=False, show_ylabel=False)
    else:
        plot_covid_visit_counts(df_visit_characteristics.visit_count_covid, show=False, show_ylabel=False)
    plt.text(0.13, 1, '(F)', transform=ax.transAxes + translation, fontweight='bold')
    
    plt.tight_layout()
    if file_out is not None:
        plt.savefig(file_out)        
    plt.show()  

# Entire Dataset Characteristics (including held-out data)

In [ ]:
df_demographics_orig = demographics(df_events_orig)
df_visit_characteristics_orig = visit_characteristics(df_events_orig, time_window_pre, time_window_covid)

In [ ]:
race_ethnicity_orig = df_demographics_orig.race_ethnicity.value_counts()
race_ethnicity_orig = race_ethnicity_orig.reindex(index=['Hispanic', 'Asian, AIAN, or NHPI', 'Black or African American', 'White', 'Other or Unknown'])
display(race_ethnicity_orig)

In [ ]:
# Horizontal layout of plots
plot_characteristics_paper(df_demographics_orig, df_visit_characteristics_orig, figsize=(10, 7), 
                           file_out=path.join(dir_output, 'full_cohort_distributions.pdf'))

In [ ]:
sex = df_demographics_orig.sex.value_counts()
sex = sex / np.sum(sex) * 100
display(sex)

In [ ]:
# Pre-COVID number of visits - print percentages
print('pre-COVID')
hist, bin_edges = plot_precovid_visit_counts(df_visit_characteristics_orig.visit_count_pre_covid, show=False)
for i, j in zip(hist, bin_edges):
    print(f'{i / n_patients_orig * 100:.01f}: {j}')
    
# COVID number of visits - print percentages
print('\nCOVID')
hist, bin_edges = plot_covid_visit_counts(df_visit_characteristics_orig.visit_count_covid, show=False)
for i, j in zip(hist, bin_edges):
    print(f'{i / n_patients_orig * 100:.01f}: {j}')
    
plt.clf()

In [ ]:
# Vaccination rate
vax_full = vaccination_rate(df_events_orig, show=True)

In [ ]:
# COVID Severity Phenotyping
class COVIDSeverity:    
    MODERATE = 1
    SEVERE = 2
    CRITICAL = 3    
    
def cps_severity(cps):
    concepts_critical = [
        # Conditions
        434489,   # Dead
        4195694,  # ARDS
        46273390, # Dependence on respirator
        196236,   # Septic shock
        # Procedures
        2106469, # Intubation, endotracheal, emergency procedure
        2745444, # Insertion of Endotracheal Airway into Trachea, Via Natural or Artificial Opening    
        2788036, # Respiratory Ventilation, Less than 24 Consecutive Hours
        2788037, # Respiratory Ventilation, 24-96 Consecutive Hours
        2788038, # Respiratory Ventilation, Greater than 96 Consecutive Hours
    ]
    concepts_severe = [
        # Conditions
        46271075, # AHRF
        437390,   # Hypoxemia
        # Procedures - non-invasive respiratory ventilation
        1781160,
        2788018,
        2788017,
        2788016,
        2787824,
        2787823,
        2788019,
        2788020,
        2788021,
        1781161,
        2788022,
        2788023,
        2788024,
        2788025,
        2788026,
        1781162,
        2788027,
        2788028,
    ]
    for c in concepts_critical:
        if str(c) in cps.sequence.words:
            return COVIDSeverity.CRITICAL
    for c in concepts_severe:
        if str(c) in cps.sequence.words:
            return COVIDSeverity.SEVERE
    return COVIDSeverity.MODERATE

SeverityTuple = namedtuple('SeverityTuple', ['Moderate', 'Severe', 'Critical'])

patient_severity_orig = dict()
for pid, cps in cpss_orig.items():
    patient_severity_orig[pid] = cps_severity(cps)

severity_list = np.array(list(patient_severity_orig.values()))    
severity_orig_cnt = SeverityTuple(np.sum(severity_list == COVIDSeverity.MODERATE), np.sum(severity_list == COVIDSeverity.SEVERE), np.sum(severity_list == COVIDSeverity.CRITICAL))
severity_orig_pct = SeverityTuple(*[c / n_patients_orig * 100 for c in severity_orig_cnt])
print(f'Moderate COVID: {severity_orig_cnt.Moderate} / {n_patients_orig} ({severity_orig_pct.Moderate:.01f}%)')
print(f'Severe COVID: {severity_orig_cnt.Severe} / {n_patients_orig} ({severity_orig_pct.Severe:.01f}%)')
print(f'Critical COVID: {severity_orig_cnt.Critical} / {n_patients_orig} ({severity_orig_pct.Critical:.01f}%)')

# Create patient vectors for cohort   

In [ ]:
# Number of times to run inference on a patient sequence. Mean of inferred vectors will be used
n_inferences = 1

t1 = time()

# Store the patient vectors in a list. The list should have the same order as cpss
patient_vectors_orig = list()
patient_vectors_dict_orig = dict()
for i, pid in enumerate(cpss_orig):
    cps = cpss_orig[pid]    
    l_dm = list()
    l_dbow = list()
    
    # Take the mean of n_inferences iterations of inferring the vector
    for _ in range(n_inferences):
        l_dm.append(model_dm.infer_vector(cps.sequence.words))
        l_dbow.append(model_dbow.infer_vector(cps.sequence.words))
    a_dm = np.mean(np.array(l_dm), axis=0)
    a_dbow = np.mean(np.array(l_dbow), axis=0)
    
    # Concatenate DM and DBOW to form a single vector
    patient_vector = np.concatenate((a_dm, a_dbow))  
    
    # Store
    patient_vectors_orig.append(patient_vector)
    patient_vectors_dict_orig[pid] = patient_vector
    
    if i % 1000 == 0:
        ellapsed_time = (time() - t1) / 60
        print(f'{i}: {ellapsed_time:.02f} min')
        
# Convert to numpy array
patient_vectors_orig = np.array(patient_vectors_orig)

file_vectors = path.join(dir_output, 'covid_cohort_patient_vectors_orig.pkl')
with open(file_vectors, 'wb') as f:
    pickle.dump(patient_vectors_dict_orig, f, protocol=pickle.HIGHEST_PROTOCOL)

# Hold out test data

In [ ]:
n_holdout = round(n_patients_orig/10)
print(n_holdout)

## Out-of-time hold out

In [ ]:
df_events_holdout_oot = df_events_orig.iloc[-n_holdout:, :]
df_events = df_events_orig.iloc[:-n_holdout, :]
patient_ids_holdout_oot = df_events_holdout_oot.index.tolist()
print(len(df_events_holdout_oot))
print(len(df_events))

## In-time hold out

In [ ]:
r = Random(42)
patient_ids_holdout_it = r.sample(df_events.index.tolist(), n_holdout)
df_events_holdout_it = df_events.loc[patient_ids_holdout_it, :].sort_values(by='start_date')
df_events = df_events[~df_events.index.isin(patient_ids_holdout_it)]

## Split up the training vs holdout data

In [ ]:
# Combined holdout 
patient_ids_holdout = patient_ids_holdout_oot + patient_ids_holdout_it

# Data to train on
patient_ids = list(set(patient_ids_orig) - set(patient_ids_holdout))
n_patients = len(patient_ids)
cpss_pre = dict()
cpss = dict()
mask_train = np.zeros(n_patients_orig, dtype=np.bool_)
for i, pid in enumerate(cpss_orig.keys()):
    if pid in patient_ids:
        cpss_pre[pid] = cpss_pre_orig[pid]
        cpss[pid] = cpss_orig[pid]
        mask_train[i] = True        
patient_vectors = patient_vectors_orig[mask_train, :]
patient_ids = list(cpss.keys())  # keep patient_ids and patient_vectors in parallel
print(len(cpss_pre))
print(len(cpss))
print(patient_vectors.shape)

# Training Dataset Characteristics

In [ ]:
df_demographics = demographics(df_events)
df_visit_characteristics = visit_characteristics(df_events, time_window_pre, time_window_covid)

In [ ]:
race_ethnicity_full = df_demographics.race_ethnicity.value_counts()
race_ethnicity_full = race_ethnicity_full.reindex(index=['Hispanic', 'Asian, AIAN, or NHPI', 'Black or African American', 'White', 'Other or Unknown'])
display(race_ethnicity_full)

In [ ]:
plot_characteristics_paper(df_demographics, df_visit_characteristics, figsize=(10, 7), 
                           file_out=path.join(dir_output, 'full_cohort_distributions_training.pdf'))

In [ ]:
# Vaccination rate
vax_full = vaccination_rate(df_events)

In [ ]:
patient_severity = dict()
for pid, cps in cpss.items():
    patient_severity[pid] = cps_severity(cps)

severity_list = np.array(list(patient_severity.values()))    
severity_full_cnt = SeverityTuple(np.sum(severity_list == COVIDSeverity.MODERATE), np.sum(severity_list == COVIDSeverity.SEVERE), np.sum(severity_list == COVIDSeverity.CRITICAL))
severity_full_pct = SeverityTuple(*[c / n_patients * 100 for c in severity_full_cnt])
print(f'Moderate COVID: {severity_full_cnt.Moderate} / {n_patients} ({severity_full_pct.Moderate:.01f}%)')
print(f'Severe COVID: {severity_full_cnt.Severe} / {n_patients} ({severity_full_pct.Severe:.01f}%)')
print(f'Critical COVID: {severity_full_cnt.Critical} / {n_patients} ({severity_full_pct.Critical:.01f}%)')

# t-SNE using the defined disease subtypes  
this takes about 75 seconds

In [ ]:
t1 = time()

tsne = TSNE(n_components=2, metric='euclidean', perplexity=30, learning_rate=1000, init='pca', 
            n_iter=1000, n_jobs=8, verbose=2, random_state=42)
patient_vectors_tsne = tsne.fit_transform(patient_vectors)

ellapsed_time = (time() - t1) / 60
print(f'{ellapsed_time} min')

In [ ]:
# t-SNE plot
plt.figure(figsize=(16,12))
known_labels = [df_concepts.loc[cps.label, 'concept_name'] for cps in cpss.values()]
sns.scatterplot(patient_vectors_tsne[:,0], patient_vectors_tsne[:,1], 
                hue=known_labels, legend='full', palette=sns.color_palette('bright', 1))

# Clustering

### K-means

#### Elbow method to select K

In [ ]:
# calculate distortion for a range of number of cluster
distortions = []
kmeans_dict = dict()
r = range(1, 60)
for i in r:
    km = KMeans(n_clusters=i, random_state=42, n_jobs=8)
    kmeans_dict[i] = km.fit(patient_vectors)
    distortions.append(km.inertia_)
    
    # Progress
    if (i >= 30) and (i % 5 == 0):
        print(i)
    
# Save the kmeans models
f_kmeans = f'kmeans_{datetime.now().isoformat()}.pkl'
f_kmeans = path.join(dir_output, f_kmeans)
pickle.dump(kmeans_dict, open(f_kmeans, 'wb'))      

# plot
plt.plot(r, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
ks_plot = [10, 15, 20, 25, 30]
for k in ks_plot:
    clustering = kmeans_dict[k]
    predicted_labels = clustering.predict(patient_vectors)    
    palette_kmeans = sns.color_palette('bright', k)
    plt.figure(figsize=(20,10))
    sns.scatterplot(patient_vectors_tsne[:,0], patient_vectors_tsne[:,1], 
                    hue=predicted_labels, legend='full', palette=palette_kmeans,
                    style=predicted_labels, markers=['o']*10+['X']*10+['v']*10+['d']*10+['s']*10+['P']*10)  

In [ ]:
# specify the desired model
n_clusters = 20
clustering = kmeans_dict[n_clusters]
predicted_labels = clustering.predict(patient_vectors)

# Full Cohort Profile

In [ ]:
# Build a list of patient sequences for each known subtype
patients_known_disease_subtype = defaultdict(list)
for cps in cpss.values():
    patients_known_disease_subtype[cps.label].append(cps)

# dict[disease subtype concept_id] => dict[domain_id] => dict[concept_id] => Counter
known_disease_profiles = dict()

# For each disease subtype, count the number of patients each concept is observed in
for known_disease_subtype, cohort_cpss in patients_known_disease_subtype.items():
    # dict[domain_id] => Counter
    cohort_domain_concept_counter = defaultdict(Counter)
    
    for cohort_cps in cohort_cpss:
        # cohort_cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
        # Convert it to a list of concept_ids (ints)
        seq = [int(x) for x in cohort_cps.sequence.words]
        
        # Keep track of which conpepts we've already seen for this patient so we don't add again
        concepts_observed = list()  
        
        for concept in seq:
            if concept in concepts_observed:
                # Already seen this concept for this patient
                continue            
                
            domain = df_concepts.loc[concept, 'domain_id']
            cohort_domain_concept_counter[domain][concept] += 1
            concepts_observed.append(concept)
            
    known_disease_profiles[known_disease_subtype] = cohort_domain_concept_counter    

##### Pre

In [ ]:
# Build a list of patient sequences for each known CKD subtype
patients_known_disease_subtype_pre = defaultdict(list)
for cps in cpss_pre.values():
    patients_known_disease_subtype_pre[cps.label].append(cps)

# dict[disease subtype concept_id] => dict[domain_id] => dict[concept_id] => Counter
known_disease_profiles_pre = dict()

# For each disease subtype, count the number of patients each concept is observed in
for known_disease_subtype, cohort_cpss in patients_known_disease_subtype_pre.items():
    # dict[domain_id] => Counter
    cohort_domain_concept_counter = defaultdict(Counter)
    
    for cohort_cps in cohort_cpss:
        # cohort_cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
        # Convert it to a list of concept_ids (ints)
        seq = [int(x) for x in cohort_cps.sequence.words]
        
        # Keep track of which conpepts we've already seen for this patient so we don't add again
        concepts_observed = list()  
        
        for concept in seq:
            if concept in concepts_observed:
                # Already seen this concept for this patient
                continue            
                
            domain = df_concepts.loc[concept, 'domain_id']
            cohort_domain_concept_counter[domain][concept] += 1
            concepts_observed.append(concept)
            
    known_disease_profiles_pre[known_disease_subtype] = cohort_domain_concept_counter    

### Learned Disease Subtypes

In [ ]:
def subtype_profiling(cpss, predicted_labels):
    # Build a list of patient sequences for each predicted (learned) disease subtype
    patients_learned_disease_subtype = defaultdict(list)
    for i, cps in enumerate(cpss.values()):
        patients_learned_disease_subtype[predicted_labels[i]].append(cps)

    # dict[disease subtype concept_id] => dict[domain_id] => dict[concept_id] => Counter
    learned_disease_profiles = dict()

    # For each disease subtype, count the number of patients each concept is observed in
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
        # dict[domain_id] => Counter
        cohort_domain_concept_counter = defaultdict(Counter)

        for cohort_cps in cohort_cpss:
            # cohort_cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
            # Convert it to a list of concept_ids (ints)
            seq = [int(x) for x in cohort_cps.sequence.words]

            # Keep track of which conpepts we've already seen for this patient so we don't add again
            concepts_observed = list()  

            for concept in seq:
                if concept in concepts_observed:
                    # Already seen this concept for this patient
                    continue            

                domain = df_concepts.loc[concept, 'domain_id']
                cohort_domain_concept_counter[domain][concept] += 1
                concepts_observed.append(concept)

        learned_disease_profiles[disease_subtype] = cohort_domain_concept_counter    
        
    return patients_learned_disease_subtype, learned_disease_profiles

In [ ]:
# Demographics = namedtuple('Demographics', ['age', 'sex'])

def subtype_demographics(patients_learned_disease_subtype, df_demographics):
    subtype_demogs = dict()
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
        person_ids = [x.person_id for x in cohort_cpss]
        subtype_demogs[disease_subtype] = df_demographics.loc[df_demographics.index.isin(person_ids), :]
    return subtype_demogs                

def subtype_visit_profiles(patients_learned_disease_subtype, df_visit_characteristics):
    subtype_visit_profs = dict()
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
        person_ids = [x.person_id for x in cohort_cpss]
        subtype_visit_profs[disease_subtype] = df_visit_characteristics.loc[df_visit_characteristics.index.isin(person_ids), :]
    return subtype_visit_profs

def subtype_primary_discharge_diagnoses(patients_learned_disease_subtype, df_events):
    subtype_discharge_diagnoses = dict()
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
        person_ids = [x.person_id for x in cohort_cpss]
        df_events_subtype = df_events.loc[df_events.index.isin(person_ids), :]
        visit_occurrence_ids = [str(x) for x in df_events_subtype.visit_occurrence_id.tolist()]
        
        # Reduces the weight of a PDD if a patient has multiple PDDs
        sql = f'''WITH pdd AS (SELECT person_id, co.condition_concept_id
        FROM condition_occurrence co
        WHERE co.condition_status_concept_id = 32903 -- primary discharge diagnosis
            AND visit_occurrence_id IN ({','.join(visit_occurrence_ids)})
        ),

        pc AS (SELECT person_id, 1.0/count(*) AS weight FROM pdd GROUP BY person_id)

        SELECT c.concept_id, c.concept_name, sum(weight) AS count
        FROM pdd
        JOIN pc ON pdd.person_id = pc.person_id 
        JOIN concept c ON pdd.condition_concept_id = c.concept_id
        GROUP BY c.concept_id, c.concept_name
        ORDER BY count DESC;
        '''

        df_discharge_diag = pd.read_sql(sql, engine)
        n_persons = len(person_ids)
        df_discharge_diag['percent'] = df_discharge_diag['count'] / n_persons * 100
        subtype_discharge_diagnoses[disease_subtype] = df_discharge_diag
    return subtype_discharge_diagnoses

def subtype_vaccination(patients_learned_disease_subtype, df_events):
    vaccination = dict()
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():
        person_ids = [x.person_id for x in cohort_cpss]
        df_events_subtype = df_events[df_events.index.isin(person_ids)]
        vaccination[disease_subtype] = vaccination_rate(df_events_subtype)
    return vaccination

In [ ]:
patients_learned_disease_subtype, learned_disease_profiles = subtype_profiling(cpss, predicted_labels)

In [ ]:
patients_learned_disease_subtype_pre, learned_disease_profiles_pre = subtype_profiling(cpss_pre, predicted_labels)

In [ ]:
subtype_demogs = subtype_demographics(patients_learned_disease_subtype, df_demographics)

In [ ]:
subtype_visit_profs = subtype_visit_profiles(patients_learned_disease_subtype, df_visit_characteristics)

In [ ]:
subtype_discharge_diagnoses = subtype_primary_discharge_diagnoses(patients_learned_disease_subtype, df_events)

In [ ]:
subtype_vaccinations = subtype_vaccination(patients_learned_disease_subtype, df_events)

In [ ]:
def subtype_severe_covid(patients_learned_disease_subtype):
    severity_cnts = dict()
    severity_pcts = dict()
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():                
        severities = np.array([patient_severity[cps.person_id] for cps in cohort_cpss])
        n_subtype = len(cohort_cpss)
        n_moderate = np.sum(severities == COVIDSeverity.MODERATE)
        n_severe = np.sum(severities == COVIDSeverity.SEVERE)
        n_critical = np.sum(severities == COVIDSeverity.CRITICAL)
        severity_cnt = SeverityTuple(n_moderate, n_severe, n_critical)
        severity_cnts[disease_subtype] = severity_cnt
        severity_pcts[disease_subtype] = SeverityTuple(*[n / n_subtype * 100 for n in severity_cnt])
    return severity_cnts, severity_pcts

subtype_severity_cnts, subtype_severity_pcts = subtype_severe_covid(patients_learned_disease_subtype)

#### Re-arrange groups labels

In [ ]:
def severity_score(severity_pct_tuple):
    return severity_pct_tuple.Moderate*1 + severity_pct_tuple.Severe*2 + severity_pct_tuple.Critical*3

# Sort the subtypes by severity phenotype
# severity_scores = [subtype_severity_pcts[i].Moderate/100*1 + subtype_severity_pcts[i].Severe/100*2 + subtype_severity_pcts[i].Critical/100*3 for i in range(n_clusters)]
severity_scores = [severity_score(subtype_severity_pcts[i]) for i in range(n_clusters)]
subtype_severity_index_to_cluster_index = {i+1:x for i,x in enumerate(np.argsort(severity_scores, axis=0))}
subtype_cluster_index_to_severity_index = {v:k for k,v in subtype_severity_index_to_cluster_index.items()}
predicted_labels_renamed = [subtype_cluster_index_to_severity_index[i] for i in predicted_labels]

In [ ]:
subtype_cluster_index_to_custom_index = subtype_cluster_index_to_severity_index
subtype_custom_index_to_cluster_index = subtype_severity_index_to_cluster_index
predicted_labels_renamed = [subtype_cluster_index_to_custom_index[i] for i in predicted_labels]

In [ ]:
# Show the t-SNE with new labels (this was generated with LOS median, 75%, 25%)
palette_kmeans = sns.color_palette('bright', 10)
# Color spectrum order
palette_kmeans = [palette_kmeans[i] for i in [3, 1, 8, 2, 9, 0, 4, 6, 5, 7, 3, 1, 8, 2, 9, 0, 4, 6, 5, 7]]  
plt.figure(figsize=(13,8))
sns.scatterplot(patient_vectors_tsne[:,0], patient_vectors_tsne[:,1], 
                hue=predicted_labels_renamed, legend='full', palette=palette_kmeans,
                style=predicted_labels_renamed, markers=['o']*10+['X']*10+['v']*10+['d']*10+['s']*10+['P']*10,
                alpha=0.6)  
plt.savefig(path.join(dir_output, 'tsne.pdf'))

# Tables

In [ ]:
def sig_style(sig, reverse=False):
    sty = 'color: black'
    if reverse:
        sig = sig * -1
    if sig < 0:
        sty = 'font-weight: bold; color: blue'
    elif sig > 0:
        sty = 'font-weight: bold; color: red'
    return sty

def sig_style_binary(sig):    
    if sig:
        return 'font-weight: bold; color: black'
    else:
        return 'color: black'

def _chi(x_subtype, n_disease_subtype, x_full, n_patients, alpha, complement=True):
    obs_sub = np.array([x_subtype, n_disease_subtype - x_subtype])
    obs_comp = np.array([x_full, n_patients - x_full])
    if complement:
        obs_comp -= obs_sub
    obs = np.array([obs_sub, obs_comp])
    _, p, _, _ = chi2_contingency(obs)
    return p < alpha

def _chi_str(x_subtype, n_disease_subtype, x_full, n_patients, alpha, complement=True):
    sig = _chi(x_subtype, n_disease_subtype, x_full, n_patients, alpha, complement)
    return '*' if sig else ''

def _chi_style(x_subtype, n_disease_subtype, x_full, n_patients, alpha, complement=True):
    sig = _chi(x_subtype, n_disease_subtype, x_full, n_patients, alpha, complement)
    p_sub = x_subtype / n_disease_subtype
    p_full = x_full / n_patients
    return sig_style(sig * 1 if p_sub > p_full else -1)
    

def _chi_freq(f_subtype, f_full, alpha, complement=True):
    if complement:
        f_full = f_full - f_subtype
    obs = np.array([f_subtype, f_full])
    _, p, _, _ = chi2_contingency(obs)
    return p < alpha

def _chi_freq_style(f_subtype, f_full, alpha, complement=True):
    sig = _chi_freq(f_subtype, f_full, alpha, complement)
    return sig_style_binary(sig)

def _med_iqr(x, ref=None, alpha=None, complement=True):
    q = np.percentile(x, [50, 25, 75])
    s1 = f'{q[0]:.01f}' 
    s2 = f'[{q[1]:.01f}, {q[2]:.01f}]'
    sig = 0
    if ref is not None and alpha is not None:
        if complement:
            # Assume ref contains x in it. Remove each instance of x
            ref = list(ref)
            for xi in x:
                ref.remove(xi)
        
        # Mann-Whitney U Test against ref
        p = mannwhitneyu(x, ref)[1]
        if p < alpha:
            ref_med = np.percentile(ref, [50])
            if q[0] < ref_med[0]:
                sig = -1
            elif q[0] > ref_med[0]:
                sig = 1
            else:
                # medians are the same. compare means to differentiate
                sig = -1 if np.mean(x) < np.mean(ref) else 1
                    
    return s1, s2, sig

def _med_iqr_h(x, ref=None, alpha=None, complement=True):
    s1, s2, sig = _med_iqr(x, ref, alpha, complement)
    s = s1 + ' ' + s2
    if sig:
        s += '*'
    return s

def _med_iqr_v(x, ref=None, alpha=None, style=False, complement=True):
    s1, s2, sig = _med_iqr(x, ref, alpha, complement)
    if style:
        return s1 + '\n' + s2, sig_style(sig)
    else:
        return s1 + ('*' if sig else '') + '\n' + s2

def _med_iqr_dates(x, ref=None, alpha=None, complement=True):
    x = x.astype('datetime64[ns]')
    q = x.quantile([0.5, 0.25, 0.75], interpolation="linear")
    s1 = f'{str(q[0.5])[:10]}' 
    s2 = f'[{str(q[0.25])[:10]},'
    s3 = f'{str(q[0.75])[:10]}]'
    sig = 0
    if ref is not None and alpha is not None:
        ref = ref.astype('datetime64[ns]')
        ref_med = ref.quantile([0.5], interpolation='linear')
        ref = list(ref)
        if complement:
            # Assume ref contains x in it. Remove each instance of x
            ref = list(ref)
            for xi in x:
                ref.remove(xi)
                
        # Mann-Whitney U Test against ref
        p = mannwhitneyu(list(x), ref)[1]
        if p < alpha:
            
            if q[0.5] < ref_med[0.5]:
                sig = -1
            else:
                sig = 1            
    return s1, s2, s3, sig

    
def _med_iqr_dates_h(x, ref=None, alpha=None, style=False):
    s1, s2, s3, sig = _med_iqr_dates(x, ref, alpha)
    s = ' '.join([s1, s2, s3])
    if style:
        return s, sig_style(sig)
    else:
        if sig:
            s += '*'
        return s    

def _med_iqr_dates_v(x, ref=None, alpha=None, style=False):
    s1, s2, s3, sig = _med_iqr_dates(x, ref, alpha)
    if style:
        return '\n'.join([s1, s2, s3]), sig_style(sig)
    else:
        if sig:
            s1 += '*'
        return '\n'.join([s1, s2, s3])

## Table 1

In [ ]:
name_count = 'Count'
name_age = 'Age'
name_sex = 'Sex (female)'
name_eth_race = '\n'.join(['Race-ethnicity'] + list(race_ethnicity_full.index))
name_vax = 'Vaccinated (%)'
name_start = 'Hospital start date'
name_los = 'Hospital length (days)'
name_visits_covid = 'Number of visits'
name_visits_pre = 'Number of prior visits'
names = [name_count, name_age, name_sex, name_eth_race, name_vax, name_start, name_los, name_visits_covid, name_visits_pre]

In [ ]:
# Create a summary statistics table 
df_summary = pd.DataFrame(data=None, index=names, columns=['Full']+[x for x in range(1, n_clusters+1)])
df_style = pd.DataFrame(data='color: black', index=names, columns=['Full']+[x for x in range(1, n_clusters+1)])

# Full COVID cohort
s_count = f'{n_patients} (100%)'
s_age = _med_iqr_v(df_demographics.age)
n_female_full = np.sum(df_demographics.sex == "FEMALE")
p_sex_full = n_female_full / n_patients * 100
s_sex = f'{n_female_full} ({p_sex_full:.01f}%)'
s_race_ethnicity = '\n'.join([''] + [f'{p} ({p / n_patients * 100:.01f}%)' for p in race_ethnicity_full])
n_vax_full = round(vax_full * n_patients / 100)
s_vaccinated = f'{n_vax_full} ({vax_full:.01f}%)' 
s_visit_start_date = _med_iqr_dates_v(df_visit_characteristics.event_start_date)
s_visit_length = _med_iqr_v(df_visit_characteristics.event_visit_los)
s_num_visits_covid = _med_iqr_v(df_visit_characteristics.visit_count_covid)
s_num_visits_pre_covid = _med_iqr_v(df_visit_characteristics.visit_count_pre_covid)
df_summary.loc[:, 'Full'] = [s_count, s_age, s_sex, s_race_ethnicity, s_vaccinated, s_visit_start_date, s_visit_length, s_num_visits_covid, s_num_visits_pre_covid]

# Subtypes
sty = 'color: black;'
alpha = 0.05 / 1000
for disease_subtype in range(1, n_clusters+1):
    cluster_index = subtype_custom_index_to_cluster_index[disease_subtype]
    subtype_demog = subtype_demogs[cluster_index]
    subtype_visit_prof =  subtype_visit_profs[cluster_index]
    patients_subtype = patients_learned_disease_subtype[cluster_index]
    
    n_disease_subtype = len(patients_subtype)
    s_count = f'{n_disease_subtype} ({n_disease_subtype / n_patients * 100:.01f}%)'
    s_age, sty_age = _med_iqr_v(subtype_demog.age, ref=df_demographics.age, alpha=alpha, style=True)    
    n_female = np.sum(subtype_demog.sex == "FEMALE")
    s_sex = f'{n_female} ({(n_female / n_disease_subtype * 100):.01f}%)' 
    sty_sex = _chi_style(n_female, n_disease_subtype, n_female_full, n_patients, alpha=alpha)    
    race_ethnicity = subtype_demog.race_ethnicity.value_counts().reindex(index=race_ethnicity_full.index, fill_value=0)    
    s_race_ethnicity = '\n'.join([''] + [f'{p} ({p / n_disease_subtype * 100:.01f}%)' for p in race_ethnicity])    
    sty_race_ethnicity = _chi_freq_style(race_ethnicity, race_ethnicity_full, alpha)
    p_vax = subtype_vaccinations[cluster_index]
    n_vax = round(p_vax * n_disease_subtype / 100)
    s_vaccinated = f'{n_vax} ({p_vax:.01f}%)'
    sty_vaccinated = _chi_style(n_vax, n_disease_subtype, n_vax_full, n_patients, alpha=alpha)
    s_visit_start_date, sty_visit_start_date = _med_iqr_dates_v(subtype_visit_prof.event_start_date, ref=df_visit_characteristics.event_start_date, alpha=alpha, style=True)
    s_visit_length, sty_visit_length = _med_iqr_v(subtype_visit_prof.event_visit_los, ref=df_visit_characteristics.event_visit_los, alpha=alpha, style=True)    
    s_num_visits_covid, sty_num_visits_covid = _med_iqr_v(subtype_visit_prof.visit_count_covid, ref=df_visit_characteristics.visit_count_covid, alpha=alpha, style=True)
    s_num_visits_pre_covid, sty_num_visits_pre_covid = _med_iqr_v(subtype_visit_prof.visit_count_pre_covid, ref=df_visit_characteristics.visit_count_pre_covid, alpha=alpha, style=True)    
    df_summary.loc[:, disease_subtype] = [s_count, s_age, s_sex, s_race_ethnicity, s_vaccinated, s_visit_start_date, s_visit_length, s_num_visits_covid, s_num_visits_pre_covid]
    df_style.loc[:, disease_subtype] = [sty, sty_age, sty_sex, sty_race_ethnicity, sty_vaccinated, sty_visit_start_date, sty_visit_length, sty_num_visits_covid, sty_num_visits_pre_covid]

df_styler_summary_h = df_summary.style.apply(lambda _: df_style, axis=None).applymap(lambda _: 'white-space: pre-wrap').applymap_index(lambda _: 'white-space: pre-wrap')
display(df_styler_summary_h)

### Table 2. COVID-19 subgroup concept prevalences using concept sets

In [ ]:
concept_sets = [
    # ['COVID-19', [37311061]],
    ['Fever', [437663]],
    ['Cough', [254761]],
    ['Diarrhea', [196523]],
    ['Constipation', [75860]],
    ['Dyspnea', [312437]],
    ['Viral pneumonia', [261326, 3661408]],
    ['Hypoxemia', [437390]],
    ['AHRF', [46271075, 37395564]],
    ['ARDS', [4195694, 4191650]],
    ['Pleural effusion', [254061]],
    ['Atelectasis', [261880]],
    ['Anemia', [439777]],
    ['Tachycardia', [444070, 4275423, 4103295]],
    ['Heart failure', [319835, 316139, 40480603, 443580, 443587, 4242669]],
    ['ARFS', [197320, 192359]],
    ['ESRD', [193782]],
    ['Sepsis', [132797]],
    ['Septic shock', [196236]],
    ['Delirium', [373995]],
    ['Altered mental status', [436222]],
    ['Ventilation', [2106469, 2745444, 2788036, 2788037, 2788038, 46273390]],
    ['Dead', [434489]],

]

# Show the concept definitions within each concept set to double check
concept_set_names = list()
concept_set_ids = list()
for cs in concept_sets:
    concept_set_names.append(cs[0])
    concept_set_ids += cs[1]
df_concept_set_definitions = df_concepts[df_concepts.index.isin(concept_set_ids)].copy()
for cs in concept_sets:
    df_concept_set_definitions.loc[df_concept_set_definitions.index.isin(cs[1]), 'concept_set'] = cs[0]
df_concept_set_definitions.sort_values(by='concept_set', inplace=True)

# Re-order, rename, and display
df_concept_set_definitions.reset_index(inplace=True)  # show concept_id as a normal column
df_concept_set_definitions = df_concept_set_definitions.loc[:, ['concept_set', 'concept_id', 'concept_name']]
df_concept_set_definitions.columns = ['Concept Set', 'OMOP Concept ID', 'Concept Name']
display(df_concept_set_definitions)

In [ ]:
def concept_set_profiling(cpss, concept_sets):
    cs_counter = Counter()
    for cps in cpss:
        # cps.sequence is a TaggedDoc with sequence of concepts stored as strings. 
        # Convert it to a list of concept_ids (ints)
        seq = [int(x) for x in cps.sequence.words]

        # Check if this patient has any of the concepts from each of the concept sets
        for cs_name, cs_concepts in concept_sets:
            for concept in cs_concepts:
                if concept in seq:
                    cs_counter[cs_name] += 1
                    break           
                    
    return cs_counter

def subtype_profiling_concept_sets(patients_learned_disease_subtype, concept_sets):
    # dict[disease subtype] => dict[concept set (string)] => Counter
    learned_disease_profiles = dict()

    # For each disease subtype, count the number of patients each concept is observed in
    for disease_subtype, cohort_cpss in patients_learned_disease_subtype.items():                    
        learned_disease_profiles[disease_subtype] = concept_set_profiling(cohort_cpss, concept_sets)
        
    return learned_disease_profiles

full_concept_set_profiles = concept_set_profiling(cpss.values(), concept_sets)
subtype_concept_set_profiles = subtype_profiling_concept_sets(patients_learned_disease_subtype, concept_sets)

In [ ]:
def _table_concept_set_subtype(row, custom_index, df_style=None, complement=True, adjustment=10000, show_n=False):
    concept_set_name = row.name
    cluster_index = subtype_custom_index_to_cluster_index[custom_index]
    n = subtype_concept_set_profiles[cluster_index][concept_set_name] 
    n_disease_subtype = len(patients_learned_disease_subtype[cluster_index])
    per = n / n_disease_subtype * 100
    
    if show_n:
        s = f'{n} ({per:.01f}%)'
    else:
        s = f'{per:.01f}%'
        
    if df_style is not None:
        # chi-square    
        x = full_concept_set_profiles[concept_set_name]
        obs_subtype = np.array([n, n_disease_subtype - n])
        obs_comparator = np.array([x, n_patients - x])  # full cohort frequency
        if complement:
            # use complementary cohort frequency
            obs_comparator = obs_comparator - obs_subtype
        obs = np.array([obs_subtype, obs_comparator])
        _, p, _, _ = chi2_contingency(obs)

        sty = 'color: black'
        if p < (0.05 / adjustment):
            sty = 'font-weight: bold;'
            if per < (x / n_patients * 100):
                sty += 'color: blue'
            else:
                sty += 'color: red'        
        df_style.loc[concept_set_name, subtype_cluster_index_to_custom_index[cluster_index]] = sty
    
    return s

df = pd.DataFrame(data=None, index=concept_set_names, columns=['Full']+[x for x in range(1, n_clusters+1)])
df_style = pd.DataFrame(data=None, index=concept_set_names, columns=['Full']+[x for x in range(1, n_clusters+1)])

# Add concept set prevalence of the full cohort
df['Full'] = df.apply(lambda row: f'{full_concept_set_profiles[row.name] / n_patients * 100:.01f}%', axis=1)
df_style['Full'] = 'color: black'

for i in range(1, n_clusters+1):
    # Add prevalence and get styling for significance         
    df[i] = df.apply(lambda row: _table_concept_set_subtype(row, i, df_style), axis=1)

# Add N for full cohort and each subtype
df.loc['Count', :] = [n_patients] + [len(patients_learned_disease_subtype[subtype_custom_index_to_cluster_index[i]]) for i in range(1, n_clusters+1)]
# Change the order so that the "Count" row comes first
df = df.reindex(['Count'] + concept_set_names)
df_style.loc['Count', :] = 'color: black'

# Add severity - single row
row_severity = [f'{severity_full_pct.Moderate:.01f}%\n{severity_full_pct.Severe:.01f}%\n{severity_full_pct.Critical:.01f}%']
row_severity_style = ['color:black']
severity_score_full = severity_score(severity_full_pct)
alpha = 0.05 / 10000
for i in range(1, n_clusters+1):
    cluster_index = subtype_custom_index_to_cluster_index[i]
    t_severity_cnts = subtype_severity_cnts[cluster_index]
    t_severity_pcts = subtype_severity_pcts[cluster_index]    
    sig = _chi_freq(np.array(t_severity_cnts), np.array(severity_full_cnt), alpha)
    row_severity += [f'{t_severity_pcts[0]:.01f}%\n{t_severity_pcts[1]:.01f}%\n{t_severity_pcts[2]:.01f}%']
    s = 'color: black'
    if sig:
        s = 'font-weight: bold;'
        severity_score_sub = severity_score(t_severity_pcts)
        if severity_score_sub > severity_score_full:
            s += 'color: red'
        else:
            s += 'color: blue'
    row_severity_style.append(s)
df.loc['Mild-moderate\nSevere\nCritical', :] = row_severity
df_style.loc['Mild-moderate\nSevere\nCritical', :] = row_severity_style

# Apply styling
df_style = df_style.reindex(df.index)
df_styler = df.style.apply(lambda x: df_style, axis=None).applymap(lambda x: 'white-space: pre-wrap')
df_styler = df_styler.applymap_index(lambda x: 'white-space: pre-wrap')

# Save
display(df_styler)
df_styler.to_excel(path.join(dir_output, 'table-2-concept-set-prevalences-covid.xlsx'))
df_styer_table_concept_sets_covid = df_styler

### Table 3. Pre-COVID-19 subgroup concept prevalences using concept sets

In [ ]:
concept_sets_pre = [
    ["Obesity", [433736, 434005]],
    ["Essential hypertension", [320128]],
    ["Hyperlipidemia", [432867, 438720]],
    ["T2DM", [4193704, 4193704, 201826, 43531578, 37016349, 443729, 443731, 376065, 45757363, 443733]],
    ["Pregnant", [444094, 4218813, 4244438, 4307820, 4188598, 4239938, 439658, 4084768, 4094910, 4174506, 4097608, 4197245, 4132434, 4181751, 443874, 441678, 442558, 444267, 4051642, 4274955, 433864, 434484, 444461, 4185780, 4336226, 444417, 438543, 438542, 432430, 444023, 435640, 4322726, 4266517, 4242241, 4049621, 4277749, 4283690]],
    ["Constipation", [75860]],
    ["Diarrhea", [196523]],
    ["Cough", [254761]],
    ["Dyspnea", [312437]],
    ["COPD", [255573]],
    ["Pleural effusion", [254061]],    
    ["Atherosclerosis of coronary artery", [764123]],
    ["Heart disease", [764123, 313217, 314666, 4154290, 321319, 45766207, 4152384, 321588, 4270024, 319844, 4289309, 37312532] + [319835, 316139, 4229440, 40479192, 40479576, 443580, 40480602]],
    ["ARFS", [197320, 192359]],
    ["CKD", [46271022, 43531578, 443597, 45768812, 443612, 443611]],
    ["ESRD", [193782]],
    ["Vitamin D deficiency", [436070]],
    ["Immunodeficiency disorder", [433740]],
    ["Dementia", [4182210, 374888]],
    ["SOT", [42539502, 42538117, 42539698, 42537742]]
]

# Show the concept definitions within each concept set to double check
concept_set_names_pre = list()
concept_set_ids_pre = list()
for cs in concept_sets_pre:
    concept_set_names_pre.append(cs[0])
    concept_set_ids_pre += cs[1]
df_concept_set_definitions_pre = df_concepts[df_concepts.index.isin(concept_set_ids_pre)].copy()
for cs in concept_sets_pre:
    df_concept_set_definitions_pre.loc[df_concept_set_definitions_pre.index.isin(cs[1]), 'concept_set'] = cs[0]
df_concept_set_definitions_pre.sort_values(by='concept_set', inplace=True)
display(df_concept_set_definitions_pre)

In [ ]:
full_concept_set_profiles_pre = concept_set_profiling(cpss_pre.values(), concept_sets_pre)
subtype_concept_set_profiles_pre = subtype_profiling_concept_sets(patients_learned_disease_subtype_pre, concept_sets_pre)

In [ ]:
def _table_concept_set_subtype_pre(row, custom_index, df_style=None, complement=True, adjustment=10000, show_n=False):
    concept_set_name = row.name
    cluster_index = subtype_custom_index_to_cluster_index[custom_index]
    n = subtype_concept_set_profiles_pre[cluster_index][concept_set_name] 
    n_disease_subtype = len(patients_learned_disease_subtype[cluster_index])
    per = n / n_disease_subtype * 100
    
    if show_n:
        s = f'{n} ({per:.01f}%)'
    else:
        s = f'{per:.01f}%'
        
    if df_style is not None:
        # chi-square    
        x = full_concept_set_profiles_pre[concept_set_name]
        obs_subtype = np.array([n, n_disease_subtype - n])
        obs_comparator = np.array([x, n_patients - x])  # full cohort frequency
        if complement:
            # use complementary cohort frequency
            obs_comparator = obs_comparator - obs_subtype
        obs = np.array([obs_subtype, obs_comparator])
        try:
            _, p, _, _ = chi2_contingency(obs)
        except ValueError:
            print(concept_set_name)
            print(custom_index)

        sty = 'color: black'
        if p < (0.05 / adjustment):
            sty = 'font-weight: bold;'
            if per < (x / n_patients * 100):
                sty += 'color: blue'
            else:
                sty += 'color: red'        
        df_style.loc[concept_set_name, subtype_cluster_index_to_custom_index[cluster_index]] = sty
    
    return s

df = pd.DataFrame(data=None, index=concept_set_names_pre, columns=['Full']+[x for x in range(1, n_clusters+1)])
df_style = pd.DataFrame(data=None, index=concept_set_names_pre, columns=['Full']+[x for x in range(1, n_clusters+1)])

# Add concept set prevalence of the full cohort
df['Full'] = df.apply(lambda row: f'{full_concept_set_profiles_pre[row.name] / n_patients * 100:.01f}%', axis=1)
df_style['Full'] = 'color: black'

for i in range(1, n_clusters+1):
    # Add prevalence and get styling for significance         
    df[i] = df.apply(lambda row: _table_concept_set_subtype_pre(row, i, df_style), axis=1)

# Add N for full cohort and each subtype
df.loc['Count', :] = [n_patients] + [len(patients_learned_disease_subtype[subtype_custom_index_to_cluster_index[i]]) for i in range(1, n_clusters+1)]
# Change the order so that the "Count" row comes first
df = df.reindex(['Count'] + concept_set_names_pre)
df_style.loc['Count', :] = 'color: black'

# Apply styling
df_style = df_style.reindex(df.index)
df_styler = df.style.apply(lambda x: df_style, axis=None).applymap(lambda x: 'white-space: pre-wrap')

# Save
display(df_styler)
df_styler.to_excel(path.join(dir_output, 'table-3-concept-set-prevalences-pre-covid.xlsx'))
df_styer_table_concept_sets_pre = df_styler

### Supplemental Tables - Subgroup COVID and pre-COVID conditions

In [ ]:
# Show the stats of each concept that is observed in > X% of each disease subtype
print('Note: subtype percentages can add up to > 100% since different timelines can be extracted from each patient to represent a disease subtype')
adjustment = 10**6
prevalence_threshold_analysis = 10
prevalence_threshold_display = 10
prevalence_threshold_analysis_pre = 10
prevalence_threshold_display_pre = 10
count_threshold_display = 10
df_conditions_covid = None
df_conditions_precovid = None
complement = True

for disease_subtype in range(1, n_clusters+1):
    cluster_index = subtype_custom_index_to_cluster_index[disease_subtype]
    cohort_domain_concept_counter = learned_disease_profiles[cluster_index]
    cohort_domain_concept_counter_pre = learned_disease_profiles_pre[cluster_index]
    n_disease_subtype = len(patients_learned_disease_subtype[cluster_index])
    n_comparator = n_patients - n_disease_subtype if complement else n_patients
    
    for domain in ['Condition']:        
        ################################## COVID ##################################
        concept_counter = cohort_domain_concept_counter[domain]        
        
        # Create a DataFrame from the counts
        df_subtype_domain_counts = pd.DataFrame.from_dict(concept_counter, orient='index', columns=['count'])
        df_subtype_domain_counts.index.name = 'concept_id'
        
        # Calculate prevalence of each concept within the disease subtype
        df_subtype_domain_counts['subcohort prevalence'] = df_subtype_domain_counts['count'] / n_disease_subtype * 100
        
        # Add concept_name to DataFrame
        df_subtype_domain_counts = df_subtype_domain_counts.join(df_concepts['concept_name'], how='left')
        
        # Re-arrange column order,  sort by descending order of count, and display
        df_subtype_domain_counts = df_subtype_domain_counts[['concept_name', 'count', 'subcohort prevalence']]        
        df_subtype_domain_counts = df_subtype_domain_counts.loc[((df_subtype_domain_counts['subcohort prevalence'] > prevalence_threshold_analysis)), :]
        df_subtype_domain_counts['COVID-19 prevalence'], df_subtype_domain_counts['P value'], df_subtype_domain_counts['sig'] = [0, 0, 0]

        # Chi-square        
        for index, row in df_subtype_domain_counts.iterrows():
            o = row['count']
            x = known_disease_profiles[37311061][domain][index]            
            if complement:
                # compare against complementary cohort
                x -= o
            obs = np.array([[o, n_disease_subtype - o], [x, n_comparator - x]])
            g, p, dof, exp = chi2_contingency(obs)
            full_prev = x / n_comparator * 100.0
            df_subtype_domain_counts.loc[index, 'full prev'] = full_prev
            df_subtype_domain_counts.loc[index, 'COVID-19 prevalence'] = f'{x} ({full_prev:.01f}%)' 
            df_subtype_domain_counts.loc[index, 'P value'] = (f'{p:.02e}')
            df_subtype_domain_counts.loc[index, 'sig'] = p < (0.05 / adjustment)   
                
        # Filter
        display_filter = (df_subtype_domain_counts['count'] >= count_threshold_display) & \
                (df_subtype_domain_counts['subcohort prevalence'] > prevalence_threshold_display) & (df_subtype_domain_counts['sig'])
        df_subtype_domain_counts = df_subtype_domain_counts.loc[display_filter, :]                
                
        df_subtype_domain_counts['Subgroup prevalence'] = df_subtype_domain_counts['count'].apply(lambda x: f'{int(x)} ({x / n_disease_subtype * 100.0:.01f}%)')
        
        # Add the count to the beginning
        if len(df_subtype_domain_counts) > 0:
            df_subtype_domain_counts.loc[0, ['concept_name', 'count', 'Subgroup prevalence', 'COVID-19 prevalence', 'P value']] = \
                ['Count', int(n_disease_subtype), f'{int(n_disease_subtype)} (100%)', f'{n_comparator} (100%)', '']
            
        df_subtype_domain_counts.sort_values(by='count', ascending=False, inplace=True)
        df_subtype_domain_counts['Subgroup'] = disease_subtype
        
        if df_conditions_covid is None:
            df_conditions_covid = df_subtype_domain_counts
        else:
            df_conditions_covid = pd.concat([df_conditions_covid, df_subtype_domain_counts])
        
        
        ################################## pre-COVID ##################################
        concept_counter = cohort_domain_concept_counter_pre[domain]        
        
        # Create a DataFrame from the counts
        df_subtype_domain_counts = pd.DataFrame.from_dict(concept_counter, orient='index', columns=['count'])
        df_subtype_domain_counts.index.name = 'concept_id'
        
        # Calculate prevalence of each concept within the disease subtype
        df_subtype_domain_counts['subcohort prevalence'] = df_subtype_domain_counts['count'] / n_disease_subtype * 100
        
        # Add concept_name to DataFrame
        df_subtype_domain_counts = df_subtype_domain_counts.join(df_concepts['concept_name'], how='left')
        
        # Re-arrange column order,  sort by descending order of count, and display
        df_subtype_domain_counts = df_subtype_domain_counts[['concept_name', 'count', 'subcohort prevalence']]
        df_subtype_domain_counts = df_subtype_domain_counts.loc[((df_subtype_domain_counts['subcohort prevalence'] > prevalence_threshold_analysis_pre)), :]
        df_subtype_domain_counts['COVID-19 prevalence'], df_subtype_domain_counts['P value'], df_subtype_domain_counts['sig'] = [0, 0, 0]
        
        # Chi-square        
        for index, row in df_subtype_domain_counts.iterrows():
            o = row['count']
            x = known_disease_profiles_pre[37311061][domain][index]
            if complement:
                # compare against complementary cohort
                x -= o
            obs = np.array([[o, n_disease_subtype - o], [x, n_comparator - x]])
            g, p, dof, exp = chi2_contingency(obs)
            full_prev = x / n_comparator * 100.0
            df_subtype_domain_counts.loc[index, 'full prev'] = full_prev
            df_subtype_domain_counts.loc[index, 'COVID-19 prevalence'] = f'{x} ({full_prev:.01f}%)' 
            df_subtype_domain_counts.loc[index, 'P value'] = (f'{p:.02e}')
            df_subtype_domain_counts.loc[index, 'sig'] = p < (0.05 / adjustment)     
                                       
        # Filter        
        display_filter = (df_subtype_domain_counts['count'] >= count_threshold_display) & \
                ((df_subtype_domain_counts['subcohort prevalence'] > prevalence_threshold_display_pre) & (df_subtype_domain_counts['sig']))
        df_subtype_domain_counts = df_subtype_domain_counts.loc[display_filter, :]
                                  
        df_subtype_domain_counts['Subgroup prevalence'] = df_subtype_domain_counts['count'].apply(lambda x: f'{int(x)} ({x / n_disease_subtype * 100.0:.01f}%)')
        
        # Add the count to the beginning
        if len(df_subtype_domain_counts) > 0:
            df_subtype_domain_counts.loc[0, ['concept_name', 'count', 'Subgroup prevalence', 'COVID-19 prevalence', 'P value']] = \
                ['Count', int(n_disease_subtype), f'{int(n_disease_subtype)} (100%)', f'{n_comparator} (100%)', '']
        
        df_subtype_domain_counts.sort_values(by='count', ascending=False, inplace=True)  
        df_subtype_domain_counts['Subgroup'] = disease_subtype
        
        if df_conditions_precovid is None:
            df_conditions_precovid = df_subtype_domain_counts
        else:
            df_conditions_precovid = pd.concat([df_conditions_precovid, df_subtype_domain_counts])           

df_conditions_covid.rename(columns={'concept_name': 'Condition'}, inplace=True)
df_conditions_precovid.rename(columns={'concept_name': 'Condition'}, inplace=True)
if complement:
    df_conditions_covid.rename(columns={'COVID-19 prevalence': 'Complement prevalence'}, inplace=True)
    df_conditions_precovid.rename(columns={'COVID-19 prevalence': 'Complement prevalence'}, inplace=True)

In [ ]:
# Show certain columns, don't show COVID-19 (37311061)
df = df_conditions_covid[['Subgroup', 'Condition', 'Subgroup prevalence', 'Complement prevalence', 'P value', 'subcohort prevalence', 'full prev']].drop(index=37311061)
display(df.head())
df.to_excel(path.join(dir_output, 'table-subgroup-conditions-covid.xlsx'))
print(len(df_conditions_covid))

# Show certain columns
df = df_conditions_precovid[['Subgroup', 'Condition', 'Subgroup prevalence', 'Complement prevalence', 'P value', 'subcohort prevalence', 'full prev']]
display(df.head())
df.to_excel(path.join(dir_output, 'table-subgroup-conditions-precovid.xlsx'))
print(len(df_conditions_precovid))

### Supplementary Table 3. discharge diagnoses

In [ ]:
n_diag = 3
df_table_discharge_diag = pd.DataFrame(index=list(range(1, n_clusters+1)), columns=list(range(1, n_diag+1)))
for i in range(1, n_clusters+1):
    cluster_index = subtype_custom_index_to_cluster_index[i]
    df_table_discharge_diag.loc[i, :] = subtype_discharge_diagnoses[cluster_index].head(n_diag).apply(lambda r: f'{r["concept_name"]} ({r["percent"]:.01f}%)', axis=1).tolist()

display(df_table_discharge_diag)

## Evaluate hold-outs

In [ ]:
# distribution of distances of each vector to cluster center
cluster_distances_train = defaultdict(list)
for i in range(n_patients):
    cluster = predicted_labels[i]
    vector = patient_vectors[i]
    dist = np.linalg.norm(clustering.cluster_centers_[cluster] - vector)
    cluster_distances_train[cluster].append(dist)
    
for i in range(1, n_clusters+1):
    cluster_index = subtype_custom_index_to_cluster_index[i]
    ds = cluster_distances_train[cluster_index]
    q = np.quantile(ds, [.25, .50, .75, .95])
    print(f'{i} ({len(ds)}): {q}')

In [ ]:
def holdout_evaluation(patient_ids, clustering, cluster_distances_train, dist_percentile=.95, print_quantiles=True):
    df = pd.DataFrame(data=0, index=list(range(1, clustering.n_clusters+1)) + ['Full', 'P-value'], columns=['Outliers'])
    cluster_distances = defaultdict(list)
    vectors = np.array([patient_vectors_dict_orig[pid] for pid in patient_ids])
    predicted_labels = clustering.predict(vectors)
    n_patients_holdout = len(patient_ids)
    
    # Calculate distances of holdout vectors to the cluster centers
    for v, l in zip(vectors, predicted_labels):
        dist = np.linalg.norm(clustering.cluster_centers_[l] - v)
        cluster_distances[l].append(dist)
    
    # See how many distances are farther than the 95th percentile distance of each cluster from the training set
    far = np.zeros(clustering.n_clusters)
    far_train = np.zeros(clustering.n_clusters)
    n_train = 0
    for i in range(1, n_clusters+1):        
        # Get distances and count of outliers from training
        cluster_index = subtype_custom_index_to_cluster_index[i]                
        cdt_i = cluster_distances_train[cluster_index]
        dist_thresh = np.quantile(cdt_i, dist_percentile)
        far_train[i-1] = np.sum(np.array(cdt_i) > dist_thresh)
        n_train += len(cdt_i)
        
        ds = cluster_distances[cluster_index]
        n_subtype = len(ds)        
        if n_subtype == 0:
            print(f'{i}: no representation')
            df.loc[i, 'Outliers'] = '0 / 0 (NA)'
            continue
        
        if print_quantiles:
            q = np.quantile(ds, [.25, .50, .75, .95])
            print(f'{i} ({n_subtype}): {q}')
        
        # Count outliers from holdout
        f_i = np.sum(np.array(ds) > dist_thresh)
        far[i-1] = f_i        
        s = f'{f_i} / {n_subtype} ({f_i / n_subtype * 100:.01f}%)'
        df.loc[i, 'Outliers'] = s
        
    f_all = int(np.sum(far))
    f_train = int(np.sum(far_train))
    s = f'{f_all} / {n_patients_holdout} ({f_all / n_patients_holdout * 100:.01f}%)'
    print(f'all: {s}')    
    df.loc['Full', 'Outliers'] = s
    
    # chi-square    
    obs = np.array([[f_all, n_patients_holdout - f_all], 
                    [round(.05*n_patients_holdout), round(.95*n_patients_holdout)]])
    _, p, _, _ = chi2_contingency(obs)
    print(f'Full p-value: {p:.04f}')    
    df.loc['P-value', 'Outliers'] = p
    
    return df

In [ ]:
# in-time holdout
print('In-time holdout')
df_holdout_it = holdout_evaluation(patient_ids_holdout_it, clustering, cluster_distances_train, print_quantiles=False)
df_holdout_it.columns = ['In-time Holdout']
print()

# out-of-time holdout
print('Out-of-time holdout')
df_holdout_oot = holdout_evaluation(patient_ids_holdout_oot, clustering, cluster_distances_train, print_quantiles=False)
df_holdout_oot.columns = ['Out-of-time Holdout']
print()

# combined holdout
print('Combined holdout')
df_holdout_combined = holdout_evaluation(patient_ids_holdout_it + patient_ids_holdout_oot, clustering, cluster_distances_train, print_quantiles=False)
df_holdout_combined.columns = ['Combined Holdout']

df_holdout = pd.concat([df_holdout_it, df_holdout_oot, df_holdout_combined], axis=1)
display(df_holdout)

# Patient Clustering Over Time

In [ ]:
def patient_vector_inference(model_dm, model_dbow, sequence, n_inferences=1):
    # Number of times to run inference on a patient sequence. Mean of inferred vectors will be used
    n_inferences = 1
    
    l_dm = list()
    l_dbow = list()

    # Take the mean of n_inferences iterations of inferring the vector
    for _ in range(n_inferences):
        l_dm.append(model_dm.infer_vector(sequence.words))
        l_dbow.append(model_dm.infer_vector(sequence.words))
    a_dm = np.mean(np.array(l_dm), axis=0)
    a_dbow = np.mean(np.array(l_dm), axis=0)
    patient_vector = np.concatenate((a_dm, a_dbow))
    patient_vector = np.array(patient_vector)
    return patient_vector

In [ ]:
def create_patient_vector_sequences_window(f_pcs_in, df_events, model_dm, model_dbow, f_seq_out=None, time_window=[-14,28], randomize_order=True, random_seed=None, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and extracts sequences within the time_window days around the 
    first occurrence of any encountered desired concept. Creates patient vectors on a day-by-day basis using
    cummulative data from the beginning of the time window to the current date
    
    Note: save_intermediates makes it a lot slower 
    
    Return
    ------
    dict[person_id] -> list(patient_vectors)"""

    # For keeping track of processing time
    t1 = time()

    # pcss - cohort patient sequences: list of CohortPatientSeq objects
    cohort_temporal_vectors = dict()
    count = 0
    
    # Time window for finding occurrences 
    time_window_pre = timedelta(days=time_window[0])
    time_window_post = timedelta(days=time_window[1])
    
    r = Random(random_seed)
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the header line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)
            
            if pid not in df_events.index:
                # Couldn't find this person_id in the events table
                continue
                
            # Get the index date from events table    
            event_start_date = df_events.loc[pid, 'start_date']
            date_lower = event_start_date + time_window_pre
            date_upper = event_start_date + time_window_post
            
            current_seq = list()
            patient_vectors = list()
            for do in date_occurrences:
                if do.date < date_lower:
                    continue

                if do.date > date_upper:
                    # No more date_occurrences within the desired time window.                         
                    break

                # The date_occurrence is within the time_window. Add occurrences to seq
                concepts = do.concept_ids
                if randomize_order:
                    # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                    r.shuffle(concepts)
                current_seq += concepts
                
                # Convert the sequence of OMOP concept IDs to TaggedDocument for D2V processing
                tagged_doc_seq = TaggedDocument(words=[str(x) for x in current_seq], tags=[pid])   
                
                # Perform patient vector inferencing
                patient_vector = patient_vector_inference(model_dm, model_dbow, tagged_doc_seq)         
                patient_vectors.append(patient_vector)
                             

            cohort_temporal_vectors[pid] = patient_vectors

            # Display progress
            count += 1
            if count % 100 == 0:
                if verbose: 
                    # Processing time
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(cpss, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save cohort temporal vectors         
        pickle.dump(cohort_temporal_vectors, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return cohort_temporal_vectors

In [ ]:
def create_patient_vector_sequences_visit(f_pcs_in, df_events, model_dm, model_dbow, cummulative=True, f_seq_out=None, time_window=[0, 0], 
                                          randomize_order=True, random_seed=None, verbose=False, save_intermediates=False): 
    """ Reads the patient_code_sequences.txt file and extracts sequences within the patient's index event +/- time_window. 
    Creates patient vectors on a day-by-day basis using the index event from [start_date + time_window[0], end_date + time_window[1]]
    cummulative data from the beginning of the time window to the current date
    
    Note: save_intermediates makes it a lot slower 
    
    Return
    ------
    dict[person_id] -> list(patient_vectors)"""

    # For keeping track of processing time
    t1 = time()

    # pcss - cohort patient sequences: list of CohortPatientSeq objects
    cohort_temporal_vectors = dict()
    count = 0
    
    # Time window for finding occurrences 
    time_window_pre = timedelta(days=time_window[0])
    time_window_post = timedelta(days=time_window[1])
    
    r = Random(random_seed)
    
    if f_seq_out:
        f_intermediate = f_seq_out + '.tmp'
    
    # Read patient_code_sequences.txt
    with open(f_pcs_in) as fh:  
        # Skip the header line
        fh.readline()
        
        for line in fh:
            # Parse the line into person_id and list of date_occurrences
            pid, date_occurrences = _process_pcs_line(line)
            
            if pid not in df_events.index:
                # Couldn't find this person_id in the events table
                continue
                
            # Get the index date from events table    
            event_start_date = df_events.loc[pid, 'start_date']
            event_end_date = df_events.loc[pid, 'end_date']
            date_lower = event_start_date + time_window_pre
            date_upper = event_end_date + time_window_post
            
            current_seq = list()
            patient_vectors = list()
            for do in date_occurrences:            
                if do.date < date_lower:
                    continue

                if do.date > date_upper:
                    # No more date_occurrences within the desired time window.                         
                    break
                    
                if not cummulative: 
                    # do not accumulate concepts from the beginning of the time range
                    current_seq = list()

                # The date_occurrence is within the time_window. Add occurrences to seq
                concepts = do.concept_ids
                if randomize_order:
                    # Randomize the order of concepts occurring on the same date. Shuffle is applied in place
                    r.shuffle(concepts)
                current_seq += concepts
                
                # Convert the sequence of OMOP concept IDs to TaggedDocument for D2V processing
                tagged_doc_seq = TaggedDocument(words=[str(x) for x in current_seq], tags=[pid])   
                
                # Perform patient vector inferencing
                patient_vector = patient_vector_inference(model_dm, model_dbow, tagged_doc_seq)         
                patient_vectors.append(patient_vector)
                             

            cohort_temporal_vectors[pid] = patient_vectors

            # Display progress
            count += 1
            if count % 100 == 0:
                if verbose: 
                    # Processing time
                    ellapsed_time = (time() - t1) / 60
                    print(f'{count} - {ellapsed_time:.01f} min')

                if save_intermediates and f_seq_out:
                    # Save a backup copy of the data
                    pickle.dump(cpss, open(f_intermediate, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)      

    if f_seq_out:
        # Save cohort temporal vectors         
        pickle.dump(cohort_temporal_vectors, open(f_seq_out, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        # Delete the backup file
        if save_intermediates and path.exists(f_intermediate):
            os.remove(f_intermediate)

    # Display overall processing time
    ellapsed_time = (time() - t1) / 60
    print(f'{count} - {ellapsed_time:.01f} min')
    
    return cohort_temporal_vectors

### Using cummulative MCS from hospitalization
Each daily vector has data from admission up to the given date

In [ ]:
# Create patient vectors using cummulative data from the beginning of the time window up to each day
# This takes approximately 4 hours
f_seq_out = path.join(dir_output, 'daily_vectors_visitrange_cummulative.pkl')
cohort_temporal_vectors_cummulative = create_patient_vector_sequences_visit(file_pcs, df_events_orig, model_dm, model_dbow,
                                                                f_seq_out=f_seq_out, cummulative=True, randomize_order=True, 
                                                                random_seed=42, verbose=True, save_intermediates=False)  

In [ ]:
# Predict labels for each daily vector
cohort_predicted_labels_cummulative = {pid: [subtype_cluster_index_to_custom_index[i] for i in clustering.predict(vectors)] for pid, vectors in cohort_temporal_vectors_cummulative.items() if len(vectors) > 0}

### Discharge

In [ ]:
# get deaths
sql = """SELECT v.visit_occurrence_id, v.person_id
FROM covid19_table_name covid
JOIN visit_occurrence v ON covid.visit_occurrence_id = v.visit_occurrence_id
WHERE v.discharge_to_concept_id = 4216643;"""
df_visit_death = pd.read_sql(sql, engine).set_index('visit_occurrence_id')
df_visit_death = df_events.join(df_visit_death, on='visit_occurrence_id', how='inner')
display(df_visit_death)

In [ ]:
# Get "discharge to" information
sql = """WITH discharge AS (SELECT v.discharge_to_concept_id, COUNT(*) AS count
FROM covid19_table_name covid
JOIN visit_occurrence v ON covid.visit_occurrence_id = v.visit_occurrence_id
GROUP BY v.discharge_to_concept_id)

SELECT 
    cp.person_id, cp.visit_occurrence_id, v.discharge_to_concept_id, c.concept_name AS discharge_to,
    CASE
        WHEN discharge_to_concept_id IN (8536, 4021968, 4161979, 4146681, 44814650) THEN 'Discharged'
        WHEN discharge_to_concept_id IN (8863, 8920, 8970, 8717, 8971, 8676, 38004519, 8827) THEN 'Care'
        WHEN discharge_to_concept_id IN (4216643, 8546) THEN 'Death or Hospice'
        ELSE 'ERROR!!!'
    END as discharge_to_grouped
FROM covid19_table_name cp
JOIN visit_occurrence v ON cp.visit_occurrence_id = v.visit_occurrence_id
JOIN concept c ON v.discharge_to_concept_id = c.concept_id;"""
df_discharged_to = pd.read_sql(sql, engine).set_index('visit_occurrence_id')

# Only keep the visit_occurrences in df_events_orig (1st event per patient)
df_discharged_to = df_discharged_to[df_discharged_to.index.isin(df_events_orig.visit_occurrence_id)]
df_discharged_to = df_discharged_to.reset_index().set_index('person_id')

# Show counts of discharge_to
display(df_discharged_to.discharge_to.value_counts())
display(df_discharged_to.discharge_to_grouped.value_counts())

### Transition characterization

In [ ]:
def count_transitions(cohort_predicted_labels, df_discharged_to, n_clusters):
    state_start = 0
    state_end = n_clusters + 1
    state_care = n_clusters + 2
    state_death = n_clusters + 3
    dict_discharge_to_to_state = {
        'Discharged': state_end,
        'Care': state_care,
        'Death or Hospice': state_death
    }
        
    transition_matrix = np.zeros([n_clusters+4, n_clusters+4], dtype=np.uint16)
    duration_tracker = defaultdict(list)
    for pid, labels in cohort_predicted_labels.items():
        current_label = state_start
        duration = 1
        for label in labels:
            if label != current_label:
                # Count the transitions between states
                transition_matrix[current_label, label] += 1
                if current_label != state_start:
                    # Keep track of duration in state before transition
                    duration_tracker[(current_label, label)].append(duration)
                current_label = label
                duration = 1
            else:
                if current_label != state_start:
                    duration += 1
                
        # Add transition to final state, e.g., discharged, additional care, or death/hospice
        final_state = dict_discharge_to_to_state[df_discharged_to.loc[pid, 'discharge_to_grouped']]        
        transition_matrix[current_label, final_state] += 1
        duration_tracker[(current_label, final_state)].append(duration)                    
    
    # Get median of duration before transition between states
    duration_matrix = np.zeros([n_clusters+4, n_clusters+4], dtype=np.uint16)
    for transition, durations in duration_tracker.items():
        duration_matrix[transition[0], transition[1]] = np.median(durations)
        
    return transition_matrix, duration_matrix

In [ ]:
def count_paths(cohort_predicted_labels, df_discharge_to, n_clusters):    
    path_tracker = defaultdict(list)
    for pid, labels in cohort_predicted_labels.items():
        current_state = None
        current_duration = None
        patient_states = list()
        state_durations = list()        
        for label in labels:
            if label != current_state:
                if current_state != None and current_duration != None:
                    patient_states.append(current_state)
                    state_durations.append(current_duration)
                current_state = label
                current_duration = 1
            else:
                current_duration += 1
                
        patient_states.append(current_state)
        state_durations.append(current_duration)
        
        # Append final state, e.g., discharge, additional care, or death/hospice
        if type(df_discharge_to.loc[pid, 'discharge_to_grouped']) is not str:
            print(pid)
            print(df_discharge_to.loc[pid, 'discharge_to_grouped'])
        patient_states.append(df_discharge_to.loc[pid, 'discharge_to_grouped'])        
        state_durations.append(0)
        
        path_tracker[tuple(patient_states)].append(state_durations)
        
    return path_tracker

#### Transition characterization
only patients with primary discharge diagnoses of COVID-19, sepsis, and viral pneumonia

In [ ]:
def patients_with_covid_discharge_diagnoses(df_events):
    visit_occurrence_ids = df_events.visit_occurrence_id
    sql = f'''SELECT co.person_id
    FROM condition_occurrence co
    WHERE co.condition_status_concept_id = 32903 -- primary discharge diagnosis
        AND condition_concept_id IN (
            37311061,  -- COVID-19
            261326, -- Viral Pneumonia
            132797 -- Sepsis
        )
        AND visit_occurrence_id IN ({','.join([str(i) for i in visit_occurrence_ids])});
    '''
    return pd.read_sql(sql, engine).person_id.tolist()

In [ ]:
patient_ids_covid_disch_diag = patients_with_covid_discharge_diagnoses(df_events_orig)
n_patients_cdd = len(patient_ids_covid_disch_diag)
print(n_patients_cdd)

In [ ]:
# CDD: covid discharge diagnoses
cohort_predicted_labels_cummulative_cdd = {pid:labels for pid, labels in cohort_predicted_labels_cummulative.items() if pid in patient_ids_covid_disch_diag}
transition_matrix_cdd, duration_matrix_cdd = count_transitions(cohort_predicted_labels_cummulative_cdd, df_discharged_to, n_clusters)
col_names = ['admission'] + [str(i) for i in range(1, n_clusters+1)] + ['discharge', 'care', 'death']

df_transition_matrix_cdd = pd.DataFrame(transition_matrix_cdd, columns=col_names, index=col_names)
df_transition_matrix_cdd.to_csv(path.join(dir_output, 'transition_matrix_extra_discharge.csv'))

df_duration_matrix_cdd = pd.DataFrame(duration_matrix_cdd, columns=col_names, index=col_names)
df_duration_matrix_cdd.to_csv(path.join(dir_output, 'duration_matrix_extra_discharge.csv'))

# Make a combined transition count and duration matrix for Supplementary Table
df_trans_dur_matrix_cdd = pd.DataFrame('-', columns=col_names, index=col_names)
for i in range(n_clusters+4):
    for j in range(n_clusters+4):
        if transition_matrix_cdd[i, j] >= 5:
            df_trans_dur_matrix_cdd.iloc[i, j] = f'{transition_matrix_cdd[i, j]}, {duration_matrix_cdd[i, j]}'
df_trans_dur_matrix_cdd = df_trans_dur_matrix_cdd.drop(labels=['discharge', 'care', 'death'], axis=0).drop(labels=['admission'], axis=1)
display(df_trans_dur_matrix_cdd)
df_trans_dur_matrix_cdd.to_excel(path.join(dir_output, 'trans_dur_matrix_extra_discharge.xlsx'))

In [ ]:
# Table config
n_paths_table = 5
n_states_max = 5

path_tracker_cdd = count_paths(cohort_predicted_labels_cummulative_cdd, df_discharged_to, n_clusters)
paths_cdd = list(path_tracker_cdd.keys())
path_durations_cdd = list(path_tracker_cdd.values())
n_paths_cdd = len(paths_cdd)

# number of patients with each path
path_prevalences_cdd = [len(path_tracker_cdd[path]) for path in paths_cdd]
path_prev_argsort_cdd = np.argsort(path_prevalences_cdd)[::-1]

# how many paths had more than X patients?
n_x = 10
path_x = [p for p in path_prevalences_cdd if p >= n_x]
path_x_sum = np.sum(path_x)
print(f'There are {len(path_x)} paths with at least {n_x} patients, totaling {path_x_sum} ({path_x_sum / n_patients_cdd * 100:.01f}%) patients')

# quantiles of total duration of each path
path_quantile_total_durations_cdd = np.zeros([n_paths_cdd], dtype=[('50', 'f4'), ('75', 'f4'), ('25', 'f4')])
for i in range(n_paths_cdd):
    path_quantile_total_durations_cdd[i] = tuple(np.quantile([np.sum(d) for d in path_durations_cdd[i]], [.50, .75, .25]))
path_quantile_total_durations_argsort_cdd = np.argsort(path_quantile_total_durations_cdd, axis=0, order=['50', '75', '25'])[::-1]
path_quantile_total_durations_dict_cdd = {paths_cdd[i]:path_quantile_total_durations_cdd[i] for i in range(n_paths_cdd)}

# Mortality of base paths (base path = path composed of subgroups, i.e., excluding discharge/death)
base_paths_cdd = list({p[:-1] for p in paths_cdd})
n_base_paths = len(base_paths_cdd)
base_paths_n_disch = [len(path_tracker_cdd[p + ('Discharged',)]) for p in base_paths_cdd]
base_paths_n_care = [len(path_tracker_cdd[p + ('Care',)]) for p in base_paths_cdd]
base_paths_n_death = [len(path_tracker_cdd[p + ('Death or Hospice',)]) for p in base_paths_cdd]
base_paths_n_total = [base_paths_n_disch[i] + base_paths_n_care[i] + base_paths_n_death[i] for i in range(n_base_paths)]
base_paths_death_rate = [base_paths_n_death[i] / base_paths_n_total[i] * 100 for i in range(n_base_paths)]
base_paths_death_rate_dict = {base_paths_cdd[i]:f'{base_paths_n_death[i]} / {base_paths_n_total[i]} ({base_paths_death_rate[i]:.01f}%)' for i in range(n_base_paths)}
base_paths_death_rate_argsort = np.argsort(base_paths_death_rate)

# quantiles of total duration of each base path
basepath_quantile_total_durations_cdd = np.zeros([n_base_paths], dtype=[('50', 'f4'), ('75', 'f4'), ('25', 'f4')])
basepath_quantile_total_durations_dict_cdd = dict()
for i, p in enumerate(base_paths_cdd):
    combined_durations = path_tracker_cdd[p + ('Discharged', )] + path_tracker_cdd[p + ('Care', )] + path_tracker_cdd[p + ('Death or Hospice',)]
    duration_quantile = tuple(np.quantile([np.sum(d) for d in combined_durations], [.50, .75, .25]))
    basepath_quantile_total_durations_cdd[i] = duration_quantile
    basepath_quantile_total_durations_dict_cdd[p] = duration_quantile
basepath_quantile_total_durations_argsort_cdd = np.argsort(basepath_quantile_total_durations_cdd, axis=0, order=['50', '75', '25'])[::-1]

# Create a table of the top N paths
print(f'{len(paths_cdd)}\n')

# Paths with highest prevalence
print('Highest Prevalence')
df = pd.DataFrame(index=range(n_paths_table), columns=[f'State {i}' for i in range(1, n_states_max+1)] + ['Total Duration (days)', 'Count (%)', 'Mortality (%)'])
df.fillna('', inplace=True)
n_represented = 0
for i in range(n_paths_table):
    p = paths_cdd[path_prev_argsort_cdd[i]]
    durations = path_tracker_cdd[p]
    
    # Get the median duration of each state
    durations_median = [np.quantile([d[j] for d in durations], 0.5) for j in range(len(p))]
    
    # Show each state and duration
    len_p = len(p)
    for j in range(len_p - 1):
        df.loc[i, f'State {j+1}'] = f'{p[j]} ({durations_median[j]} days)'
    df.loc[i, f'State {len_p}'] = p[-1]
    
    # Number of patients
    n = len(path_tracker_cdd[p])
    df.loc[i, 'Count (%)'] = f'{n} ({n / n_patients_cdd * 100:.01f}%)'
    n_represented += n
    
    # Mortality
    df.loc[i, 'Mortality (%)'] = base_paths_death_rate_dict[p[:-1]]
    
    # Total duration
    d = path_quantile_total_durations_dict_cdd[p]
    df.loc[i, 'Total Duration (days)'] = f'{d[0]} [{d[2]}, {d[1]}]'
    
df_paths_highest_count = df
display(df)
print(f'number patients represented: {n_represented} / {n_patients_cdd} ({n_represented / n_patients_cdd * 100:.01f}%)\n')

# Paths with lowest mortality
print('Lowest Mortality')
count = 0
df = pd.DataFrame(index=range(n_paths_table), columns=[f'State {i}' for i in range(1, n_states_max+1)] + ['Total Duration (days)', 'Count (%)', 'Mortality (%)'])
df.fillna('', inplace=True)
n_represented = 0

for i in range(n_base_paths):    
    if base_paths_n_total[base_paths_death_rate_argsort[i]] >= 50:
        p_base = base_paths_cdd[base_paths_death_rate_argsort[i]]
        p = p_base
        
        # Get the median duration of each state among base_path->discharge and base_path->care
        durations = path_tracker_cdd[p_base + ('Discharged',)] + path_tracker_cdd[p_base + ('Care',)] + path_tracker_cdd[p_base + ('Death or Hospice',)]
        durations_median = [np.quantile([d[j] for d in durations], 0.5) for j in range(len(p))]
        n = len(durations)

        # Show each state and duration
        len_p = len(p)
        for j in range(len_p):
            df.loc[count, f'State {j+1}'] = f'{p[j]} ({durations_median[j]} days)'

        # Number of patients        
        df.loc[count, 'Count (%)'] = f'{n} ({n / n_patients_cdd * 100:.01f}%)'
        n_represented += n
        
        # Total duration
        d = basepath_quantile_total_durations_dict_cdd[p]
        df.loc[count, 'Total Duration (days)'] = f'{d[0]} [{d[2]}, {d[1]}]'
        
        # Mortality
        df.loc[count, 'Mortality (%)'] = base_paths_death_rate_dict[p_base]
        
        count += 1
        
    if count >= n_paths_table:
        break
df_paths_lowest_death = df
display(df)
n_death = len(df_visit_death)
print(f'number patients represented: {n_represented} / {n_patients_cdd} ({n_represented / n_patients_cdd * 100:.01f}%)\n')
    
# Paths with highest mortality
print('Highest Mortality')
count = 0
df = pd.DataFrame(index=range(n_paths_table), columns=[f'State {i}' for i in range(1, n_states_max+1)] + ['Total Duration (days)', 'Count (%)', 'Mortality (%)'])
df.fillna('', inplace=True)
n_represented = 0

for i in range(n_base_paths - 1, -1, -1):  
    if base_paths_n_total[base_paths_death_rate_argsort[i]] >= 10:
        p_base = base_paths_cdd[base_paths_death_rate_argsort[i]]
        p = p_base
        
        # Get the median duration of each state
        durations = path_tracker_cdd[p_base + ('Discharged',)] + path_tracker_cdd[p_base + ('Care',)] + path_tracker_cdd[p_base + ('Death or Hospice',)]
        durations_median = [np.quantile([d[j] for d in durations], 0.5) for j in range(len(p))]
        n = len(durations)

        # Show each state and duration
        len_p = len(p)
        for j in range(len_p):
            df.loc[count, f'State {j+1}'] = f'{p[j]} ({durations_median[j]} days)'
#         df.loc[count, f'State {len_p}'] = p[-1]

        # Number of patients        
        df.loc[count, 'Count (%)'] = f'{n} ({n / n_patients_cdd * 100:.01f}%)'
        n_represented += n
        
        # Total duration
        d = basepath_quantile_total_durations_dict_cdd[p]
        df.loc[count, 'Total Duration (days)'] = f'{d[0]} [{d[2]}, {d[1]}]'
        
        # Mortality
        df.loc[count, 'Mortality (%)'] = base_paths_death_rate_dict[p_base]
        
        count += 1
        
    if count >= n_paths_table:
        break
df_paths_highest_death = df
display(df)
n_death = len(df_visit_death)
print(f'number patients represented: {n_represented} / {n_death} ({n_represented / n_death * 100:.01f}%)\n')
    
# Paths with longest median duration
print('Longest Duration')
count = 0
df = pd.DataFrame(index=range(n_paths_table), columns=[f'State {i}' for i in range(1, n_states_max+1)] + ['Total Duration (days)', 'Count (%)', 'Mortality (%)'])
df.fillna('', inplace=True)
n_represented = 0
for i in range(n_paths_cdd):
    p = paths_cdd[path_quantile_total_durations_argsort_cdd[i]]
    n = len(path_tracker_cdd[p])
    if n >= 10: 
        durations = path_tracker_cdd[p]

        # Get the median duration of each state
        durations_median = [np.quantile([d[j] for d in durations], 0.5) for j in range(len(p))]

        # Show each state and duration
        len_p = len(p)
        for j in range(len_p - 1):
            df.loc[count, f'State {j+1}'] = f'{p[j]} ({durations_median[j]} days)'
        df.loc[count, f'State {len_p}'] = p[-1]

        # Number of patients    
        df.loc[count, 'Count (%)'] = f'{n} ({n / n_patients_cdd * 100:.01f}%)'
        n_represented += n
        
        # Total duration
        d = path_quantile_total_durations_dict_cdd[p]
        df.loc[count, 'Total Duration (days)'] = f'{d[0]} [{d[2]}, {d[1]}]'

        # Mortality
        df.loc[count, 'Mortality (%)'] = base_paths_death_rate_dict[p[:-1]]                
        
        count += 1
        
    if count >= n_paths_table:
        break
    
df_paths_longest_duration = df
display(df)
print(f'number patients represented: {n_represented} / {n_patients_cdd} ({n_represented / n_patients_cdd * 100:.01f}%)\n')

pd.concat([df_paths_highest_count, df_paths_longest_duration, df_paths_lowest_death, df_paths_highest_death])

In [ ]:
# Create a table of the top N paths
n_paths_table = 70
n_states_max = 5
display_thresh = 10

print(f'{len(paths_cdd)}\n')

# Paths with highest prevalence
print('Highest Prevalence')
df = pd.DataFrame(index=range(n_paths_table), columns=[f'State {i}' for i in range(1, n_states_max+1)] + ['Count (%)', 'Total Duration (days)', 'Mortality (%)'])
n_represented = 0
for i in range(n_paths_table):
    p = paths_cdd[path_prev_argsort_cdd[i]]
    n = len(path_tracker_cdd[p])
    
    if n < display_thresh:
        break
    
    durations = path_tracker_cdd[p]
    
    # Get the median duration of each state
    durations_median = [np.quantile([d[j] for d in durations], 0.5) for j in range(len(p))]
    
    # Show each state and duration
    len_p = len(p)
    for j in range(len_p - 1):
        df.loc[i, f'State {j+1}'] = f'{p[j]} ({durations_median[j]} days)'
    df.loc[i, f'State {len_p}'] = p[-1]
    
    # Number of patients    
    df.loc[i, 'Count (%)'] = f'{n} ({n / n_patients_cdd * 100:.01f}%)'
    n_represented += n
    
    # Mortality
    df.loc[i, 'Mortality (%)'] = base_paths_death_rate_dict[p[:-1]]
    
    # Total duration
    d = path_quantile_total_durations_dict_cdd[p]
    df.loc[i, 'Total Duration (days)'] = f'{d[0]} [{d[1]}, {d[2]}]'
    
df = df.dropna(how='all').fillna('')
df_paths_highest_count = df
display(df)
print(f'number patients represented: {n_represented} / {n_patients_cdd} ({n_represented / n_patients_cdd * 100:.01f}%)\n')

### How many patients died when their first day was in a given state

In [ ]:
alpha = 0.05 / n_clusters

# General death 
n_death_gen = df_discharged_to.discharge_to_grouped.value_counts()['Death or Hospice']
obs_death_gen = np.array([n_death_gen, n_patients_orig - n_death_gen])
print(f'General Mortality: {n_death_gen} / {n_patients_orig} ({n_death_gen / n_patients_orig * 100:.01f}%)')

# Characterize percent of deaths by state on first day 
df_admission_state_death = pd.DataFrame(index=['Full'] + list(range(n_clusters)), columns=['Death or Hospice', 'P-value'])
df_admission_state_death.index.names = ['Group']
df_admission_state_death.loc['Full', :] = [f'{n_death_gen}/{n_patients_orig} ({n_death_gen/n_patients_orig*100:.01f}%)', '']
state_lowest = 1
start_state_deaths = np.zeros(n_clusters, dtype='uint16')
start_state_counts = np.zeros(n_clusters, dtype='uint16')
count = 0
for p, d in path_tracker.items():
    n = len(d)
    count += n
    start_state_counts[p[0] - state_lowest] += n
    if p[-1] == 'Death or Hospice':
        start_state_deaths[p[0] - state_lowest] += n
print(count)
     
count = 0
for i in range(n_clusters):
    d = start_state_deaths[i]
    t = start_state_counts[i]
    if t > 0:
        # chi-square 
        obs_death_subtype = np.array([d, t-d])
        obs = np.array([obs_death_subtype, obs_death_gen - obs_death_subtype])
        _, p, _, _ = chi2_contingency(obs)
        
        df_admission_state_death.loc[i+state_lowest, ['Death or Hospice', 'P-value']] = [f'{d}/{t} ({d/t*100:.01f}%)', (f'{p:4.04f}' if p >= 0.0001 else '<0.0001') + ('*' if p < alpha else '')]
        count += 1
df_admission_state_death.dropna(inplace=True)
display(df_admission_state_death)